In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import mlb
import json
import gc

In [ ]:
# load model
import joblib
model_path = '../input/new-model/'
model1 = joblib.load(model_path+'new_model1.pkl')
model2 = joblib.load(model_path+'new_model2.pkl')
model3 = joblib.load(model_path+'new_model3.pkl')
model4 = joblib.load(model_path+'new_model4.pkl')

In [ ]:

def merge_nested_form(form, column):
  result = pd.DataFrame()
  for index, value in form[column].items():
    value = str(value)
    if (value!='nan') and (value is not np.nan):
      unpack = pd.read_json(value)
      result = pd.concat([result,unpack], axis=0)
  index = pd.Series(list(range(0,result.shape[0])))       # 给整张表重置一下index
  result = result.set_index(index)
  return result


# def merge_nested_form(form, name):
#   l = []
#   rows = len(form[name])
#   print(name)
#   print('rows:',rows)
#   for i in range(rows):
#     # for now if nan, we skip, NEED TO CHANGE later
#     print('type:',type(form[name].iloc[i]))
# #     print('ele:',form[name].iloc[i])

    
#     json_str = str(form[name].iloc[i])
#     print(json_str == 'nan')
# #     print('str:',json_str)
#     if(json_str=='nan' or pd.isna(json_str)):
#       continue
#     l.append(pd.read_json(json_str))
#   mergedDF = pd.concat(l, ignore_index = True)
#   return mergedDF

def unpack_certain_column_and_concat_date(form, column):
  form = form.copy()
  result = pd.DataFrame()                              #先创建一个空白表格
  for index, value in form[column].iteritems():        #把这一列逐行遍历
    if pd.isnull(index):                               #注意判断方法
      continue
    else:
      unpack = pd.read_json(value)                     #把有内容的单元解压缩成df
      date = form['date'].loc[index]                   #找到 unpack 出的这个表格对应的日期是多少
      rows_amount = unpack.shape[0]                    #看一下这个 unpack 的表格一共有多少行
      unpack['date'] = pd.Series(data = [int(date)] * rows_amount)
          #因为整张 unpack 表都对应一个日期，所以用乘法的方法，把这个日期广播到 date 那一列的每一行
      result = pd.concat([result, unpack], axis=0)     #把解压缩出来的内容和每个拼在一起
  #刚才为了实现广播这个操作，date的格式是int
  #现在需要把 date 那一列的格式改成 str，因为其他表 date 那一列都是 str，不改的话后面 merge 很麻烦
  #但是需要先转成 datetime 格式，因为年份月份和日期之间需要加上分界线
  result['date'] = pd.to_datetime(result['date'],format='%Y%m%d')
  result['date'] = result['date'].dt.date
  result['date'] = result['date'].astype(str)          
  index = pd.Series(list(range(0,result.shape[0])))    # 给整张表重置一下index
  result = result.set_index(index)
  return result

In [ ]:
def unique_record(data,column1,column2,new_column):
  data = data.copy()
  data[column1] = data[column1].astype(str)
  data[column2] = data[column2].astype(str)
  data[new_column] = data[column1]+data[column2]
  # 到此为止，给原表重新创造了一列，后面要以这一列为基准

  temp = data.groupby(new_column).max()             # 这里应该是 groupby().max()，这个函数好像是没保存，跑一下如果报错了就是这个有问题

  # 把一天之内参加了多场比赛的球队/队员的信息保留最大值
  # 之所以保留最大值，是因为这样也能保留下字符串的列
  # 但是这一步会导致 index 的错乱，同时 string 的列会被自动删除
  temp[new_column] = temp.index
  reset_index = pd.Series(list(range(0,temp.shape[0])))    # 给整张表重置一下index
  temp = temp.set_index(reset_index)
  # 因为刚才的 groupby 计算搞乱了 index，需要重置一下
  '''
  categorical_columns = data[data.columns[data.dtypes == object]] 
  # 这一步就很牛逼了：把数据格式为 object 的列拎出来
  # 注意，被拎出来的列中一定包括 columns1，column2，和 new_column
  result = pd.merge(temp,categorical_columns,on=new_column,how='left')
  '''
  # 因为刚才的 groupby 函数使用的是 max，不会对字符串的列产生影响，否则还要想办法处理这些列
  # 不过写在这里的处理方法也不是完美的，会导致一部分的数据被迫丢失
  result = temp
  return result

In [ ]:
def merge_two_nested_form(primary_form,secondary_form,key):
  primary_form = primary_form.copy()
  secondary_form = secondary_form.copy()
  result = pd.DataFrame()
  merge_area = round(primary_form.shape[0]/100) # 每次 merge 1%
  start = 0
  for n in range(0,100):
    start = n*merge_area
    end = min(start+merge_area,primary_form.shape[0])
    temp = pd.merge(primary_form.iloc[start:end], secondary_form, 
                    on=key, #加这个参数很重要，不然的话电脑需要一个一个去匹配，内存会爆
                    how="left")
    result = result.append(temp)
    gc.collect()
  return result

In [ ]:
def target_encode(data,y, a=0, p=0):
  # 输入的 data 格式全部都是 category value
  data = data.copy()
  columns = data.columns.drop(y)                                                        
  for n in columns:
    temp = data.dropna(subset=[n])   
        # 必须先复制一个表（没用copy是因为dropna默认inpalce为否），然后把groupby函数括号中的那一列（也就是n那一列）中的nan全都给删掉，然后再按照n那一列groupby
        # 不然会报错，因为groupby的时候是自动dropna的，transform 返回结果的行数就少了，和原来的列表行数不匹配
        # 因为pandas要求transform返回的series行数必须和原表的行数相同，即使不把transform的结果插入原表，系统也这么要求自己
        # 注意，是 transform 结果的长度不匹配，不是插入新列的时候长度不匹配。所以我们先得到transform的结果，在按照index插入原表就不存在行数不匹配的问题                                  
    data[f'{n}(target)'] = temp.groupby(n)[y].transform(lambda x: (np.sum(x)+a*p)/(x.count()+a))
        # 按照每一列 groupby，然后把每一种值对应的y给整理出来，求和 和 计数 分别作为分子分母
        # a 和 p 是两个超参数，用于避免 overfitting，默认为0
  data.drop(columns = list(columns), inplace=True)
  return data

In [ ]:
# dict_path = '/kaggle/input/encoded-dict/'
# encode_dict = json.load(open(dict_path+"dict.txt"))
encode_dict = {'playerId': {'628317': 3.572679062519813, '547989': 4.097064433099979, '519317': 14.799762667739191, '607625': 1.273158865961782, '592547': 0.3053678858514164, '641553': 0.9898237558571998, '606959': 0.18773746492715238, '435263': 2.200154062292159, '502179': 0.1294632727704282, '493200': 0.8820140120494955, '605490': 1.046546757676012, '517369': 0.537131564478753, '543939': 3.3166009508202747, '460077': 0.6123053095786506, '543768': 2.6527629427243973, '476454': 2.77548442961736, '542979': 1.3553361054030415, '461865': 0.7643633088436447, '641857': 1.5878816259261341, '664208': 0.39034472189832853, '572998': 0.1692301843471732, '519203': 7.80484160404877, '453568': 3.2681546445941403, '608701': 0.6326696045756744, '593700': 0.7168771149646599, '425772': 0.78833201810647, '592200': 0.8191229316204246, '571760': 1.6232633171161306, '592811': 0.31578419568314386, '542583': 2.661135411536607, '593144': 0.5088780682020392, '594795': 0.4038038171314761, '544928': 0.5345141124238157, '641531': 1.5710002382056245, '641490': 0.14864774030827752, '642180': 1.8938715905504047, '641432': 1.318481888299282, '570799': 0.76077051302014, '622097': 0.2441641741078269, '477132': 12.384353706874833, '446334': 4.125371511673067, '452104': 0.6293713976340802, '622761': 0.9900137626569121, '501571': 1.1159718773487715, '608349': 1.1125237506793504, '459429': 0.6460249279468859, '571718': 1.187751204860829, '642162': 0.5901590649515736, '614177': 2.6353957261975975, '457918': 4.804424508190058, '605195': 0.5902616512153042, '594992': 0.0799368390254118, '448281': 1.6622556864190603, '434538': 1.3127389385651638, '435079': 2.809047330163991, '516782': 2.8692113308230867, '592879': 0.7391042456616465, '502671': 6.30724763413386, '548389': 2.177472626976539, '488681': 0.28692344629087385, '623364': 0.40030465901010265, '592340': 0.2752096605598718, '553878': 1.377804853154436, '461314': 5.290460807558067, '592387': 1.1818175188798532, '627894': 0.5081541815622093, '643338': 1.6867843210940363, '642571': 0.12413138834921157, '425877': 6.64505751082315, '605508': 0.36608805124714555, '501659': 0.5622350193190392, '460269': 0.4007671175484125, '571946': 1.2567710153054448, '593423': 1.360678762327474, '465657': 1.073031202746122, '592169': 0.6843900975296582, '547004': 1.0616989617533723, '501817': 0.38503709538718206, '605538': 0.842415380925716, '453943': 4.646258535485707, '570240': 0.8472663593688581, '545338': 0.19592388778308764, '500743': 1.264426175884453, '605253': 0.7870372199550424, '641645': 0.962365504747945, '543807': 7.187688533689595, '624577': 8.277249969926455, '622491': 2.966402311811155, '607455': 0.3083179710346354, '488671': 1.2883698196845919, '581527': 0.8034668020432588, '624133': 0.3615185284474311, '660623': 0.5254284076277331, '519455': 1.5656770406121445, '455119': 0.8219048144177732, '645261': 1.6483611335385209, '607680': 3.6917326862484012, '605164': 2.313868630379781, '543228': 1.6041238830239974, '622713': 0.14320754179048634, '474832': 2.197335879019556, '453172': 0.8958163408529556, '571666': 2.9293900313201853, '606299': 1.5869254383810143, '605288': 0.952480897652225, '592454': 1.5814773285011545, '433587': 4.287589006931669, '592863': 0.3222774802603094, '444432': 1.1487422546726695, '460026': 0.7576407993968194, '596001': 1.104563750704543, '518516': 7.485926363808945, '592791': 2.3408501532627137, '596115': 3.2878414874440964, '571918': 0.3536831023672568, '501985': 2.3190386816702677, '596057': 1.3047048895904434, '504379': 0.507086928735178, '570663': 0.7415560488892567, '553869': 0.6296931180308978, '543243': 4.173815409733851, '657205': 0.16228600277368854, '570482': 4.413664895625509, '493247': 0.4107266168885861, '518466': 0.45718999075458067, '502239': 1.3890136102637571, '594807': 2.2758345399530295, '592518': 14.001575890270207, '547912': 0.58133816814815, '502190': 1.637294687411225, '543148': 0.44234990004735336, '608648': 0.5258751186643374, '430935': 4.890517173482893, '543557': 1.3836610028762697, '448609': 0.4236346961359337, '543308': 0.6260319767410539, '547973': 5.889538007382979, '600303': 2.190704913479175, '462382': 0.3124684802198503, '592130': 0.23589264917889566, '606424': 0.5819888833500371, '542947': 0.5389187702013144, '606160': 0.5805083941311812, '624585': 2.7216803804881557, '596019': 8.031303048531493, '543339': 1.4385143466152022, '645302': 2.1311183935963895, '592426': 0.8181608285072812, '621453': 0.9691457057614059, '457759': 6.369322615961911, '435622': 1.7243296517186593, '543685': 6.3830581175109815, '519390': 1.1195243491901135, '596112': 0.59200077138031, '475582': 2.5039842678345834, '527038': 2.1200518793832335, '446386': 0.43330159339283564, '640461': 0.9705594146262712, '642008': 0.3722367899504179, '451661': 0.4101769871376477, '572008': 0.8315203476102793, '571951': 0.42356916814879003, '664056': 2.8869054197144632, '543302': 0.517486895094054, '571467': 0.16109001112470786, '607043': 2.802664302185517, '607219': 0.6109062610659076, '656941': 5.253489724313521, '571875': 1.436375100270803, '543877': 2.0738247047140597, '543935': 0.5321638224388935, '592773': 1.046632814122266, '570560': 1.3278584518178413, '641835': 0.37469746514822233, '664948': 0.09638265464784693, '467008': 1.2712767083280665, '572761': 4.581361335723047, '500208': 1.5589972603138018, '571561': 0.4258509964546902, '643289': 1.2317064651204697, '493316': 3.569231101273552, '592351': 1.6359804461782135, '621112': 0.3955673761907123, '622497': 0.13680297069561168, '641838': 0.39094458755777556, '542364': 0.4791339719246233, '453064': 2.875992571880854, '605486': 0.60112358828682, '621261': 0.7495488186736179, '518792': 4.046984697937929, '458675': 0.6756482175601828, '656945': 0.39670081335026125, '643327': 1.0567327122123134, '621043': 4.962219902990795, '502381': 0.5278671331481135, '501981': 3.7832580470021626, '622534': 1.47456924154081, '570731': 2.6602548821055834, '445213': 0.9222224382740736, '489446': 1.6601545423448585, '605244': 0.5574998870502317, '475253': 2.239462544314813, '450203': 4.313288190340479, '642086': 2.1875509420120074, '600986': 0.1922667590732682, '660422': 0.24321245716057732, '594694': 0.6584071807146336, '592261': 1.6421337270502887, '592665': 0.5083685261260712, '543305': 4.675200654590697, '543376': 0.6562014199950563, '608596': 0.8343379625083005, '609280': 5.257090751319649, '571437': 0.8420531813949993, '592866': 1.922912742484382, '660670': 11.511047000171752, '605242': 1.4633481266722206, '471911': 2.8426921343651252, '592229': 0.641470004336876, '622110': 1.2367686126543924, '543331': 0.12585183477866133, '571510': 2.4444553818231913, '642558': 0.5566152066949815, '608369': 5.612897578704586, '571841': 0.25331823570250517, '608601': 0.1142559083583688, '502082': 0.599649055518475, '607352': 0.752297623756251, '607752': 0.8095270067723545, '669257': 2.021069292589082, '656629': 3.430255787436909, '624424': 3.660494495001961, '520471': 2.1801487508166684, '514913': 0.5749339199139198, '491624': 0.31062819351302495, '455759': 0.49500869086436144, '543208': 0.7336807166076688, '639373': 0.5205204828085108, '472610': 0.47993591988173623, '593140': 0.5028831691196184, '400085': 8.825325333911907, '543063': 2.698777036703389, '543475': 1.4811486902893012, '448802': 0.8111312332000371, '503556': 2.881507062301665, '608070': 4.258651879736105, '542454': 1.3130908977680353, '434658': 1.0892503358990817, '592761': 1.2550867305024391, '595284': 0.6563853664786747, '518626': 8.608486636332604, '502624': 2.0503088171943817, '502083': 0.30762792833714225, '607188': 0.5115524101119174, '621114': 0.18014558704479805, '519058': 4.88710688840583, '643603': 1.4078266283384533, '606131': 0.5378629945867448, '544727': 1.416804804658489, '433589': 0.6292128541451049, '514917': 1.9728492381428828, '608348': 1.1144979498926804, '663734': 0.3251732185524052, '448602': 1.398094453077613, '501896': 2.381761030734645, '518633': 1.3964739205305443, '622092': 0.4105224710697216, '608385': 1.728556891529449, '519306': 1.337246397816818, '527048': 1.9686093058910104, '596103': 1.0554644953055827, '641924': 0.4130662441145399, '592789': 5.784123027943317, '593934': 2.52922582225544, '465041': 1.68523629679439, '502212': 0.21841019616304666, '608577': 1.9316549722000476, '451584': 1.3198421551498716, '572228': 5.981892707704849, '624507': 0.21769978612890847, '571676': 0.10038462057302713, '518614': 1.485543797707605, '642584': 0.30841609123151564, '641941': 0.97437505925963, '572750': 0.29692014732784244, '665487': 8.704714229568555, '501697': 0.07554949561986932, '614179': 0.22639953899969253, '592767': 1.8402890905760838, '543506': 0.2106955357701766, '502188': 2.62461639201222, '518452': 0.6517145156209719, '571704': 1.4882775503358119, '596146': 2.8732289827948008, '458681': 3.927886210090963, '462136': 1.1535624675149307, '571616': 0.10448438531835247, '518886': 5.5401764498640205, '543699': 2.128068115023056, '502110': 10.99980411428803, '641360': 0.20100881429341205, '435221': 0.885719778457156, '451596': 0.6854782100736961, '624512': 1.0936595446384072, '545361': 23.082918669484126, '543022': 1.8602061878735605, '488810': 0.10644301008140508, '605154': 0.6858759110234717, '572863': 0.5879155516410887, '666915': 1.60608347786179, '670462': 0.03275994826534452, '656252': 0.7457291581951118, '663423': 1.076047452293033, '543883': 0.40930104317132665, '666211': 0.9307260681033579, '663776': 0.595577781564527, '621008': 0.09703399040384338, '656887': 0.6326713137154534, '612672': 1.1347142618479216, '544365': 0.5217520073790678, '621446': 1.649567371295366, '658305': 0.23179367960514327, '644337': 0.19676378332539687, '545358': 1.0390852369753423, '664314': 0.813149248436248, '663905': 0.215465923850253, '641796': 1.1130812871933145, '605119': 1.421800911442544, '630263': 0.14484022156913973, '663647': 1.203674596173541, '641513': 0.5040171277497337, '657610': 0.4064303266871177, '591712': 0.19508599007402708, '666137': 0.2690703083795331, '641958': 0.16079942526328322, '643396': 1.2332503001068524, '663886': 0.5982029859668789, '657612': 0.5081141687594698, '591693': 0.2862737872879572, '621592': 0.1287933979923667, '650859': 0.6160949966202368, '681911': 0.2648143715540994, '665152': 0.5686324884578938, '543903': 0.21211190368914418, '623149': 0.32392795196849833, '657228': 0.20180222317068844, '664641': 0.25027971681610545, '670370': 0.3456856202277816, '501213': 0.158641183215566, '666129': 0.14022759989618988, '606992': 0.44999258445776896, '572403': 0.2878254085678029, '605520': 0.18987489021642043, '623430': 0.3123392254698545, '642736': 0.22910766829907805, '600887': 0.15221110722240377, '663978': 2.05057222578912, '543056': 0.22697379944064794, '670351': 0.20306628646205707, '644433': 0.261338973863172, '641684': 0.48673090981596856, '641712': 1.747474497268857, '666619': 0.04699760294240942, '669203': 1.3816725043704674, '476883': 0.6514049969632465, '641907': 0.22795155264546457, '624413': 8.261640140663678, '642130': 0.2626847223282505, '657277': 0.853709577985529, '458584': 0.3512018590398952, '641302': 0.45362665771304317, '664714': 0.18532515730815832, '595897': 0.4724377849678206, '491696': 0.3173610781254064, '493472': 0.1543206715435042, '640448': 0.24789975982963575, '665947': 0.16830179336697162, '608678': 0.6374050883717419, '621573': 0.6842019958434932, '602104': 0.37039291613482606, '656814': 0.22896463028742858, '656685': 0.27533564847878494, '600858': 0.16075842092866313, '664869': 0.23044423962358412, '596035': 0.1977551819014379, '656738': 0.06243045887757438, '593576': 0.6731534194297754, '641778': 1.3081967802687242, '592680': 0.28822991428128064, '623381': 0.569334175974195, '592858': 0.6213590419993967, '663527': 0.291805693201774, '607391': 0.5374116895690123, '542340': 2.34153704552145, '600524': 1.4746190745011587, '670990': 0.1813879496515073, '605474': 0.17834353038254108, '624418': 0.25980166878782374, '680430': 0.018458485955626033, '543258': 0.17722218109666307, '608328': 0.3830821861577647, '642721': 0.36734867624692974, '663586': 2.4852160817945417, '660761': 0.41965861861455367, '656977': 0.3490193247402753, '663959': 0.034988994307297426, '658648': 0.1169692016937787, '665621': 0.22541320339295093, '640464': 0.6585124884534646, '475054': 0.5352173868397871, '594311': 0.22048422485254573, '595928': 0.3499904834903395, '665862': 0.5454053640840243, '519068': 0.18234913210787693, '668665': 0.12060065922730102, '663878': 1.6172068813313434, '462480': 0.08922920943558145, '605121': 0.43802198321290864, '571539': 0.1527612778162017, '592407': 0.395436658940615, '656913': 0.10523032467009778, '622065': 0.5604260480418105, '663411': 0.2560399596734911, '671277': 0.45193746610052304, '543089': 0.5833681651414264, '669432': 0.5056956980892027, '519393': 0.6027621633629536, '645307': 0.21342019481273172, '672695': 0.6357765492149532, '542946': 0.13960115550855237, '491646': 0.16185400780145304, '572308': 0.07811417978575089, '669622': 0.025005752397485453, '647336': 3.155506627871857, '621514': 0.830217646571461, '669330': 0.0662378877518289, '660271': 14.26926393130983, '527055': 0.6487180339648532, '627500': 0.2122938628607054, '591741': 0.7205632838401137, '669135': 0.06620124889864984, '622797': 0.12056440122027234, '462515': 0.09136903648606, '605255': 0.12384946925486516, '446099': 0.6169062845740191, '592865': 0.48550653263077165, '672578': 0.16283222206115672, '664850': 0.15566615451459018, '593527': 0.1517071239947058, '656240': 0.12913999071402615, '657611': 0.12117678890268857, '664852': 0.038420298170272336, '607468': 0.5690682928874539, '642233': 0.2203774753904387, '660825': 0.08892758990457503, '664126': 0.38259659595031154, '641154': 1.133063284245933, '683232': 0.06724634721542808, '658551': 0.2692258911146544, '594902': 0.30199973359080984, '518653': 0.17492884258917932, '666120': 1.4605069862243878, '676709': 0.10277921641422665, '606996': 0.20132266885516625, '571710': 0.6638711575208649, '641598': 1.5889553914041075, '676831': 0.12298180806932645, '656353': 0.17416206475123713, '606149': 0.9337545559772034, '595939': 0.15288520692894314, '621005': 0.5118054867443785, '672773': 0.4343029943369913, '642731': 1.0673323418062708, '663698': 1.0311323137012676, '543734': 0.27589546200705545, '668942': 0.5912139107370784, '543677': 0.06914424266477899, '660731': 0.31339219886543385, '660620': 0.6581115177594188, '650911': 0.12203668049310328, '605447': 0.4983550438547355, '643524': 0.26658168292606516, '623205': 0.5506182630350278, '672386': 0.6542579596869015, '663903': 1.3240452667872347, '622446': 0.6608884470427958, '572180': 0.3822727234392874, '455139': 1.901999382961011, '665742': 7.432058376687005, '641541': 0.7309368095425927, '607555': 0.2102626340394872, '622569': 0.3840829373102331, '647315': 0.09590633895564207, '552640': 0.37427863167942227, '650496': 0.1408144519235941, '613564': 0.2894700140669664, '666185': 1.5164676516615139, '658431': 0.26534426728841004, '680702': 0.0955341998155808, '571863': 0.36264794408925516, '657117': 0.14236660478913724, '621450': 0.22639824949217674, '664926': 0.14508666616590354, '656981': 0.04320963505434503, '456051': 0.2776587919629298, '656577': 0.636259008266386, '644428': 0.09647458801399693, '657434': 0.46489890675307505, '660600': 0.21451249545506362, '543193': 0.27872863288828625, '668663': 0.25503241821302153, '624503': 1.0629082169564013, '573113': 0.1468205427553324, '663330': 0.3053855003940667, '650331': 0.33100165066045895, '592473': 0.6013312664157646, '605156': 0.4618592652059295, '670623': 0.5432946155324769, '453203': 0.5765310288250476, '518876': 1.101353846801961, '572125': 0.14027100523333944, '608339': 0.307416178333213, '605388': 0.42088244277307957, '594985': 0.08602240755418775, '658069': 0.5146523903152643, '664875': 0.07027666261403916, '592779': 0.3291170369778624, '605513': 1.053891105234984, '656541': 0.5989616207490918, '643275': 0.06671174448383027, '668676': 1.4829728193702139, '663611': 1.3634895011049875, '596117': 0.43966090076418385, '643466': 0.14054903614508382, '608032': 0.2614466818319174, '592499': 0.16946518018973805, '642226': 0.04558335274595207, '605612': 0.5769111392862717, '656686': 0.5842834088602004, '570666': 1.1963803089502765, '435064': 0.29755329276407017, '543001': 0.3917843267677678, '571595': 0.26798043601297544, '643418': 0.8861972956531676, '593833': 0.46895961969760747, '657011': 0.2599440134549797, '670955': 0.14807291714893517, '670090': 0.05505058438496041, '453268': 0.7487825560225161, '608703': 0.47799221437785916, '595032': 0.1452414182885844, '643256': 0.688499027951353, '657024': 0.4306837318381517, '453281': 1.1956965060032654, '621397': 0.2734353697563103, '519301': 0.16478860268600043, '628450': 0.24718599807354544, '663630': 0.1899500510343452, '572182': 0.09765666777136735, '624419': 0.21258876486860548, '656382': 0.040902395522096124, '467100': 1.5191572210472737, '602083': 0.13235859935576855, '650644': 0.8714492030806625, '543351': 0.1782069866629198, '669211': 0.34223251636751795, '677960': 0.4374399155977479, '492802': 0.26250404204120126, '656004': 0.05929942509442038, '664054': 0.26267979691107335, '650671': 0.20448649361173965, '664042': 0.3820116595811445, '571800': 0.4412090737614625, '572383': 0.38784625301179526, '656371': 0.815066662185041, '664139': 0.17085558364212253, '543776': 0.2535656334352694, '641576': 0.16208821144844368, '614173': 1.3566131931366034, '607257': 0.22583774090309466, '592769': 0.14410727181300176, '656271': 0.2633861685763567, '641658': 0.7718112478446882, '670764': 0.07718183464545784, '458676': 0.6813618454218432, '642121': 0.2666929293287761, '473879': 0.7101609669393234, '656288': 1.0420053470413861, '641793': 0.3302444830215153, '597113': 0.1597862175641767, '622098': 0.42567878757809874, '621011': 0.3929206296159205, '500135': 0.8861180622243756, '621097': 0.5093615963631989, '669735': 0.11645378474819867, '542289': 0.09006802340467744, '670032': 0.7166362583445164, '669064': 0.2838805454019531, '602115': 0.5300949266291348, '620453': 0.337040607248957, '656976': 0.5157470932292039, '501303': 0.9708488189465312, '446321': 0.4649560745722343, '656509': 0.7477318373661942, '617228': 0.20628407900023116, '663460': 0.2959760243162282, '458677': 1.384392421479485, '657108': 1.3065278379848528, '665001': 0.26769260792124444, '607179': 0.11036887830105861, '623485': 0.23375075227733283, '664761': 1.3139170958948283, '460060': 0.22895494192844937, '502043': 2.0078677272352556, '642231': 0.4137698971590725, '657053': 1.174669587875784, '607215': 0.3479304074688634, '641680': 0.3209244001579064, '663986': 0.1433024814615174, '571858': 0.1925795591489238, '669288': 0.3402404339752076, '642607': 0.34691943724684976, '670036': 0.1559086352154764, '663556': 0.2697475232658112, '664045': 0.2935681617608673, '656793': 0.41443932968518915, '573262': 3.3894644814488797, '664062': 1.6515789947217423, '676840': 0.06743049908689355, '488846': 0.35770384915108744, '621006': 0.6482682440872822, '664057': 0.49888963214580057, '621248': 0.057383793751865135, '668941': 0.3447208064717575, '672701': 0.21799536256700303, '670550': 0.1760115772209369, '650556': 0.5428588556421893, '605479': 0.19928243912055638, '591672': 0.037339536560682826, '666808': 0.057437091517612886, '502117': 0.4255117945436122, '547170': 0.6566505752949726, '571657': 1.2204108967000808, '595222': 0.2762469297783906, '664041': 0.409504310782795, '670280': 0.2802009853250045, '666674': 0.07358288503198203, '570481': 0.7850883482891139, '605509': 0.3722418371723386, '595956': 0.3622816520780698, '676606': 0.5723382942372645, '600921': 0.09210547127635078, '622256': 0.16672737399370502, '622668': 0.06456487519205693, '624647': 0.7150880049254207, '628318': 0.3055363156896498, '663855': 2.4761414195108493, '670426': 0.23517556748582008, '641584': 0.567871849748198, '605170': 1.49822722978868, '657585': 0.1297564767813031, '660431': 0.2050948221509496, '573204': 0.24901365484483898, '656308': 2.0872685386871015, '641505': 0.7561559625596024, '615698': 0.890143133867169, '605155': 0.5730559880908533, '575070': 0.10139804522780221, '669374': 1.9965153452123356, '595981': 0.4638146893009162, '502578': 0.694957302194166, '676051': 0.16339013771680966, '598284': 0.6671090883494415, '573166': 0.02821101920560773, '657786': 0.10789190975476184, '642207': 0.7346603375847064, '669087': 0.25331872660374527, '643269': 0.34968381803264464, '663531': 0.8210682173112321, '664901': 0.4831856911888746, '518586': 1.9055673774413489, '607776': 0.6949947188316734, '642073': 0.20280653479643992, '656546': 0.9406168010358882, '458708': 0.9942229376336401, '592620': 0.45062966076524663, '606132': 0.9386475995934032, '650402': 9.469104228139223, '596043': 0.7582244767155273, '506703': 0.29958737682736336, '606625': 0.4855114498749369, '471865': 2.521246649767231, '445197': 0.19177322692326546, '605483': 5.447490538971145, '594988': 0.9291109726873734, '572971': 5.357054667943441, '457727': 2.706166597275594, '595751': 1.3659359783148692, '493329': 3.335725985639527, '461325': 1.1818362284668757, '595281': 2.2945943042604697, '599096': 0.4066988389461217, '641341': 0.3509987510715864, '622554': 0.7121397720656538, '134181': 3.766990154075621, '467827': 1.4672711756540922, '669720': 1.0800739934419135, '458015': 5.220615453979209, '543401': 2.6609784572167405, '592836': 1.8401161009239917, '622194': 0.5334064156600659, '596059': 2.5322022787722207, '642083': 0.38326665028167733, '571578': 3.6034195872227763, '605439': 0.26798964397004305, '429664': 6.2331673058294275, '596133': 1.8665629543254387, '458006': 0.4925846210808766, '521230': 2.073575375479304, '547749': 0.3300035717898904, '448614': 0.20630035695529514, '605254': 0.23282801973694842, '609275': 2.212228153944082, '430832': 4.8654536003454805, '456078': 0.76723914611877, '461829': 2.489569041654114, '546318': 1.905830187913828, '592325': 1.2146343994249387, '640460': 0.420741189597302, '641583': 0.9340536568428925, '630111': 0.49922068161089905, '542932': 0.556849553083269, '506433': 9.061709822282735, '592669': 3.0031163587917775, '606965': 0.6084873857687866, '594986': 0.487629168046699, '621121': 2.2197680286573096, '602922': 0.451938031111185, '592206': 5.039317033983736, '547982': 1.036740797664123, '641933': 2.3533939804666364, '598265': 4.763200991781353, '605540': 2.1332170254222205, '502593': 0.2813722229908884, '621439': 2.955741079243243, '431148': 0.8310518816880614, '455104': 1.0298032538269417, '593647': 0.5237630181690076, '435559': 2.346189031642958, '502327': 0.5588124691392183, '592444': 1.2415890506229705, '453284': 0.7410615048691706, '534812': 0.18962921478152636, '605135': 1.160343940537484, '622251': 0.384596506803257, '445276': 3.567138498980395, '605452': 0.9655647959106277, '643217': 5.239588868557301, '543829': 2.6133971268241765, '458731': 5.7953303547034665, '453214': 1.0878983158461017, '641871': 0.5501303859521773, '593958': 2.76955275640358, '645277': 4.818744967324767, '641501': 0.07454520702255742, '444489': 0.723059937161138, '606115': 1.6327637386191678, '506747': 0.5345278787930766, '547379': 1.2601886265258873, '518397': 0.9895325848571263, '572143': 0.4959002693059946, '607457': 0.6281510975512448, '571740': 3.0582695068889887, '571945': 2.743823968334745, '545341': 2.8735195710535946, '571521': 0.32578852489839444, '607237': 1.7586173119130122, '608718': 1.0874964274416385, '445926': 1.0129492657829153, '602074': 0.40635580195394677, '431145': 2.5502368670945827, '502042': 3.030950877756294, '596129': 1.9685587061990124, '544931': 5.719723916523254, '452095': 1.4605748086783577, '608379': 2.345175683150218, '429722': 0.9275459541860901, '554430': 4.033332706747947, '543118': 0.5991052652631076, '520980': 0.9403995032223292, '572365': 0.7840967909200245, '605412': 2.1974541288858327, '624428': 1.3121837376919947, '595373': 0.2809960321522325, '453923': 0.5357983731672421, '656185': 1.1227853293372838, '502273': 0.6489125540730577, '570632': 1.5458056307582857, '460576': 1.8502795535573988, '518617': 0.7308376362674999, '445055': 1.4148822660600668, '595879': 9.974087160024535, '475174': 2.2720290085107364, '624414': 1.1751402087942635, '456701': 2.255251100396856, '641319': 0.3929939911434695, '642003': 0.49733360914516406, '640447': 0.9092629858025487, '467055': 3.3890034305823873, '502517': 3.030240638590898, '623454': 0.3380618064929154, '546990': 1.0633823874852613, '649557': 1.7588989853161234, '542208': 0.2986243222342976, '519096': 0.3187027740179452, '462101': 1.8946954141381023, '543272': 1.6964249315081399, '453265': 0.8182223491293494, '584171': 1.1075026157901113, '605548': 0.8075960189459157, '641470': 0.6076437330167073, '547007': 0.19483083419705002, '645801': 1.6078276542404495, '593428': 5.121104267213484, '596143': 0.12623748089666761, '452254': 4.636226689917831, '501925': 1.417559942987319, '593372': 2.3112663320442777, '664856': 0.7066682622248841, '491676': 0.7678414596727944, '458913': 0.35624852364339427, '572020': 4.940761643585654, '282332': 7.189658183423214, '452657': 5.134414479362817, '502051': 0.08849992323799126, '605131': 1.795238650445396, '452678': 2.646372505495737, '519299': 1.5273387673277876, '446481': 0.8349128287500366, '643493': 0.4619892129587022, '594828': 1.613963018471208, '621471': 0.43925337682924864, '572227': 0.07919263228343562, '594953': 0.7728224319145042, '622795': 0.2490665685557766, '572287': 1.7003373581012238, '595943': 0.6274201196101731, '592567': 1.4240480972920024, '575929': 4.293772569926188, '592346': 0.6048870977362597, '516770': 2.621783210536654, '623167': 0.49684701775149287, '594577': 0.7008959496706196, '596142': 7.4304009177872405, '576397': 1.5536313631488479, '408314': 1.6842972620458858, '570256': 1.4800600628302625, '607229': 0.7936846681838658, '595918': 0.9741177121454895, '656305': 3.0676373756815893, '543281': 1.6461505308287663, '592332': 2.448535287180721, '503449': 0.37135832576773875, '408252': 1.4442330793912357, '502154': 2.4940074518844724, '657077': 4.304526019465991, '543766': 0.55216245316165, '543606': 2.7222042179343573, '541640': 0.6279617954525369, '628452': 1.0860266390440445, '570257': 0.2470136537341837, '448179': 3.1686783398875407, '279571': 0.1808457696696604, '547179': 1.936584851245552, '573009': 0.12509129558549123, '620982': 0.11545278441023894, '628338': 1.0438943746204392, '605141': 17.743261192126102, '543543': 1.573472056795651, '430945': 2.807783771726335, '608566': 1.7922122040996398, '548357': 0.1379789625997378, '595777': 2.5185395036058016, '598264': 1.232071135234364, '621244': 2.746418619416145, '598271': 0.9496414749212948, '667498': 0.4433687518718013, '518960': 2.260676293857516, '446263': 1.2128772882251702, '545363': 0.2573963941824346, '607430': 0.4332944781006581, '592450': 18.30147916689039, '578428': 2.5255257288648973, '450306': 1.6606814593070112, '501593': 0.6859268510657094, '608716': 0.3649822448295808, '642082': 0.7724234944625538, '456713': 0.9604392879471417, '456501': 2.3067702475693173, '592662': 2.4621067671210772, '571697': 2.271067537830664, '623352': 3.2376489181727774, '500871': 1.992273433822228, '611093': 0.23731038937714474, '596748': 2.4410735389881055, '455755': 0.23760921787115036, '521692': 2.274524935364299, '476589': 0.7246405107534253, '519043': 1.1375718268633166, '542303': 6.6663002643389255, '592647': 0.39741195055147627, '607461': 1.2111513414240296, '519242': 6.611598360908263, '594798': 9.703695308112682, '621111': 5.249373532861385, '592222': 0.6291961945531395, '573124': 0.8607015847971067, '461858': 0.6662533929799698, '595191': 0.7024038061878255, '606466': 2.741124436591348, '572114': 0.2596476548910621, '592804': 0.5915566472148999, '488721': 0.611141167477655, '641771': 0.7577107337417305, '621028': 1.084385352082115, '457705': 6.3372286091759, '640449': 4.069933009946518, '571788': 2.7664540127535115, '457454': 0.6959099399187622, '452655': 1.0156194812579429, '571466': 1.1843963680798144, '606192': 2.275599808127352, '572362': 0.5546612582782663, '571976': 2.498752500231348, '407812': 1.7335830233364604, '605151': 1.4003032251101977, '608638': 0.47826075692406383, '408045': 2.9139728773089537, '476451': 0.8957555010935551, '502004': 0.3356394908239053, '594777': 2.0599983471547048, '500779': 2.5324945808870662, '454560': 0.358808025433581, '600968': 0.45837464456475313, '519144': 3.56332650266797, '572888': 1.3065886779359352, '571431': 1.9543650693667953, '527054': 2.4276757276365126, '600301': 0.30415208377186814, '547888': 5.463514762118478, '591720': 1.5066035358980532, '518553': 1.178959839997653, '502748': 0.8587553622641394, '621295': 0.4553743780414726, '594951': 0.13158942011573777, '600917': 0.6197416076495386, '621559': 0.47289356026305324, '621020': 4.59732471922253, '605218': 1.1447594728018355, '453192': 2.320335478057493, '582494': 0.25739981809442514, '592233': 0.22603002965804544, '457915': 0.2936341117821906, '516416': 2.988316976462818, '489334': 0.6971309960578264, '641745': 1.4166612440588724, '434671': 2.5197418860770733, '643230': 1.0915257983803806, '592230': 0.3984856429934665, '112526': 5.843553393212682, '547180': 20.670949066839878, '622072': 2.563008287934855, '592102': 0.9225670527729107, '457763': 4.9891460250054935, '489119': 2.5368040225613684, '500724': 0.33647561111430163, '594965': 1.078538596167633, '501381': 1.5817296719860414, '621311': 1.6875060131061244, '593334': 3.1076988782652193, '592122': 1.8259404300678155, '641703': 0.400230675958784, '642715': 1.636907634987871, '592885': 13.505949455570915, '518934': 10.442575861889335, '624636': 0.2717995175286297, '592705': 0.19567372663368912, '605498': 0.8354159849025277, '641477': 0.5299524427516081, '579328': 1.7208715101623158, '543424': 0.14889916303327877, '669738': 0.22828421384046477, '545343': 0.04035780243349368, '605280': 0.32703281710227783, '676220': 0.10940853851048472, '621289': 0.2967425476981478, '622268': 0.20762116344712062, '606273': 0.6061230618392472, '664337': 0.36118364270433395, '642578': 0.1052412892642031, '596271': 0.11945594041390005, '680963': 0.02591047824111395, '608715': 0.24244575065302065, '667465': 0.07436615028602514, '622694': 0.7496755739381424, '664871': 0.2009156670306667, '476595': 0.2785502343470596, '622226': 0.3146401295799645, '613317': 0.15652689509246037, '600944': 0.23726602890054135, '605347': 0.951380629475544, '543483': 0.3706707626555748, '675912': 0.07169781095802157, '676701': 0.05610343213322156, '606995': 0.14700309991248506, '571602': 1.3233016180631463, '518902': 0.5223110922237244, '605482': 0.3450976796236576, '669456': 3.9695273767612655, '669008': 0.12615873444853626, '623913': 0.37466896857639853, '571670': 0.7472758066446492, '622772': 0.3522356484613598, '669242': 1.6648754526246994, '683734': 1.03090073462532, '623465': 0.296602476637973, '600466': 0.16678304667579283, '465753': 0.35022119670696833, '605359': 0.3954989268921896, '642136': 0.533023615484219, '667297': 0.020525478550533705, '663992': 0.19720906384702125, '668338': 0.16227291528039797, '643511': 0.5646308937177428, '572073': 0.542520521114502, '543278': 0.36837687087213444, '650489': 0.9059210894324585, '554054': 0.051646676047956516, '661440': 0.12097286731157676, '543219': 5.169887916665062, '656970': 0.6001027074003178, '623323': 0.4582010960562628, '664789': 0.19439327007207272, '519303': 0.13696921258660505, '466320': 1.894195304122335, '458924': 0.6139813584473606, '608344': 0.49674378182218454, '641829': 0.39309360428549556, '606162': 0.34690311279024033, '606956': 0.2521967441092615, '500872': 0.08168963292949372, '592716': 0.43745786317028074, '664167': 0.1563341125398769, '643778': 0.45410159340250805, '666158': 2.981440868640378, '676424': 0.024851849260374547, '656322': 0.4798676544018281, '571974': 0.7339474840535051, '534910': 0.42371544379770604, '643335': 0.3469625491620661, '664874': 0.1776991564403483, '623515': 0.2551622594142155, '657571': 0.8007321216545037, '672911': 0.059656395728015926, '598286': 0.2519744975332488, '593993': 0.26099910749705296, '593871': 2.026060971576023, '623912': 0.7814172053521039, '453527': 0.23240963310567775, '605361': 0.6803233813494812, '573668': 0.2148156705930752, '594736': 0.26766572631348357, '658792': 0.18722301583580564, '672335': 0.33927881620322675, '642851': 0.32801157024830646, '621532': 0.3336814024986965, '621593': 0.1964946317428905, '501625': 0.6848970834567744, '594742': 0.14855692786650548, '518489': 0.8766870298425171, '622783': 0.14567727658336124, '670329': 0.20050314708102385, '622780': 0.16763249660469873, '592741': 0.759401218339184, '670042': 0.37930850506655445, '596027': 0.40316163686984874, '650333': 1.3382757440892679, '664913': 0.5115547022774896, '572233': 1.2643010918181572, '607481': 0.574622406873767, '642092': 0.12099101531411374, '656811': 0.8875675145091091, '649963': 0.1802552640312136, '664774': 0.4713317693981844, '677976': 0.9978967510760574, '446399': 0.6330172406416272, '543432': 0.4791509330033108, '518693': 0.30197719103580845, '641729': 0.6688079279563357, '656847': 0.576139190608349, '621348': 0.07593696990739147, '663757': 1.705923508828958, '605441': 0.25733530524301024, '656638': 0.2933334752483999, '592826': 1.7195345632021446, '642770': 0.4390310675496793, '656730': 0.19280542147870433, '621290': 0.08023041650000623, '596101': 0.2867884764579126, '572703': 0.37922973721620384, '642211': 0.651161910826705, '571503': 0.10320865968347209, '660294': 0.8639828237259995, '543144': 0.7339173393633912, '657376': 0.7699965549491035, '621466': 0.6588576409862622, '664180': 0.3148323604525689, '665600': 0.1855028616770636, '543309': 2.068875867909962, '596451': 0.9687345832360608, '624641': 0.4697904295192464, '668800': 0.6684632518631353, '641149': 0.21166981327883014, '660365': 0.17242922271609043, '477569': 0.18982204237983263, '640444': 0.2218084447220303, '670084': 0.2330753942917644, '593523': 0.5497510903703805, '545332': 0.26510977399088825, '605469': 0.21737297337422556, '463610': 0.14704124277576458, '571917': 0.15692339174536507, '645305': 0.3538604461391648, '643320': 0.08551182548344252, '670456': 0.2586054519280784, '641420': 0.2745574791649825, '622217': 0.08012294110028227, '621078': 0.18973875250644714, '664918': 0.1864053681407341, '519326': 1.1343318535735045, '451192': 0.3817306872999097, '605357': 0.1417650654952234, '548384': 0.22072940002389077, '621056': 0.4819590796220351, '643617': 0.16883115879127794, '605501': 0.6605895965653127, '543901': 0.767875751281077, '663662': 0.5888544362027827, '668731': 0.6145700080325086, '456781': 1.1265697002920136, '623434': 0.11316751094402551, '643478': 0.3350546088046244, '592712': 0.18790183649440131, '641999': 0.39704729600650773, '670768': 0.1726096267983712, '672779': 0.1557362357163588, '657097': 0.33624072982172776, '542963': 0.40895024255100315, '518566': 0.4377252570899627, '605138': 0.13460138751078926, '542953': 0.2116922626904559, '670167': 0.17609283872240525, '542882': 0.8096977214792735, '667670': 0.32155108341793975, '571577': 0.07438544276034205, '605463': 0.14925049346135114, '656876': 0.20237153382153691, '594027': 0.46917161021644843, '670056': 0.13908299353979503, '630105': 0.9468675487021047, '543343': 0.36403615488104574, '657145': 0.4491307388209461, '605143': 0.41328596219823843, '670950': 1.0825545774913021, '643436': 0.5080094381823885, '666801': 0.08682537301637962, '606993': 0.23980778944931566, '642098': 0.47062471499797914, '664029': 0.24845903769147867, '664092': 0.18955826152583904, '669060': 0.9875962327534396, '650895': 0.8968010039461862, '606988': 0.5943874052629845, '669212': 0.034417451286470134, '657658': 0.2319999539120263, '607644': 1.580708213270606, '664129': 0.1839367417713392, '628333': 0.3583535597445374, '571901': 0.5616298154486926, '607968': 0.10750870859960134, '664040': 1.79348423310713, '518883': 0.050626040974077184, '642165': 0.5374646077340063, '665854': 0.08419778728940926, '434778': 1.5715274165597124, '623184': 0.44698728896381823, '642068': 0.24784110095235046, '669214': 0.2464841180774274, '676969': 0.11339107573301017, '630023': 0.874900814499003, '644374': 0.8535173795707298, '621363': 0.28683868816153085, '621529': 0.1733048068824189, '605130': 0.4032411587964986, '623437': 0.0658895541289436, '672284': 1.184877875813745, '573064': 0.2338087225405881, '543592': 1.0489116346556298, '592464': 0.25059021795234493, '592644': 0.9061465439451437, '623354': 0.3661434074586471, '643615': 0.397486277951595, '623168': 0.3603509078885614, '676477': 0.22496813437171378, '621438': 0.4374681412116199, '666931': 0.1456291111612312, '592859': 0.5233346888666044, '605421': 0.5548990566959897, '621129': 0.2208304844318296, '628336': 0.23410190143195775, '661388': 0.6422637797661862, '666163': 0.06375641356299391, '666128': 0.18006011563813146, '593974': 0.4646951798346902, '670712': 0.6803577879697124, '605541': 0.461541015883288, '430641': 0.12620405588468997, '595465': 0.6523875916886891, '621098': 0.14199578619431824, '475479': 0.7731273855353689, '660875': 0.10855165692428118, '641561': 0.1638842134518875, '608665': 0.9177637514032839, '475857': 0.1690983920323967, '642152': 0.559771653439627, '664028': 0.17940450057356733, '570702': 0.1532761237428033, '641582': 0.20673371776115765, '673633': 0.7587901772042823, '642528': 1.6299313664307506, '571948': 0.22840415471060851, '600965': 0.06733690218633687, '665733': 0.11295809058307099, '676979': 1.6408096893347588, '607374': 0.7489299890629131, '650893': 0.5412407954273575, '663697': 0.8876392379520639, '664977': 0.21312316276494536, '680692': 0.10140373672828751, '606167': 0.420381109557543, '543532': 0.2248546100933551, '642397': 0.44549452796771843, '643265': 0.9302971990198191, '663616': 0.45803998633011717, '425783': 2.942312947144092, '543068': 2.3338618758065364, '621366': 1.0068915918240886, '666969': 0.7673263727649707, '669294': 0.06159990852330624, '650813': 0.12479108943867845, '605446': 1.9655823332311406, '658791': 0.1567650749771176, '676604': 0.10097876821519214, '595798': 0.20562194120511776, '624522': 0.08353217538849415, '641739': 0.2051305251050564, '640462': 1.3864783525103226, '543169': 0.2229630774473844, '598287': 0.5221789538434557, '656232': 0.2925148568120116, '543377': 0.3172572510625284, '640451': 0.5998803196501429, '542924': 0.06969809226280894, '663624': 0.9687469426257548, '596071': 0.5946418492303819, '592530': 0.348246161634605, '647351': 0.6562800105904041, '664069': 0.16224356589470237, '542585': 0.26456172514872195, '643316': 0.3322729489669224, '657140': 1.5336062950727232, '642133': 1.726524889667015, '644364': 0.3988094303537475, '534606': 0.5997269934609536, '666204': 0.19134194508127145, '650530': 0.3419790627116173, '668683': 0.23908768670823893, '571980': 1.3081790055103342, '623520': 2.423634286646427, '571851': 0.4073622140890854, '666182': 4.869336544943388, '642546': 0.4639175538426052, '543746': 0.07925902426362541, '594943': 0.09572934848043059, '680777': 0.1966408789879943, '622259': 0.28851652942558537, '570488': 0.3775847959510696, '656669': 1.037740777845348, '668670': 0.44531775936230156, '592468': 0.5871354784083933, '605335': 0.0440674219705021, '664196': 0.20610509612624453, '572021': 0.41141074685542844, '620445': 0.13613567807405383, '642456': 0.15787511946581406, '663567': 0.4284606727642385, '649966': 1.637036753817598, '592280': 0.11722533294294886, '487675': 0.14620518186761025, '675921': 0.4955995694054515, '656547': 1.0828170651297253, '607471': 0.10176564945517608, '623474': 0.20873625208219218, '656725': 0.20255643827105999, '642203': 0.227190163892084, '656464': 0.2348017991497184, '596049': 0.07966071092937953, '628451': 0.1301944995406761, '466988': 0.3597902575474649, '621458': 1.233643305119907, '572204': 2.009118670978112, '605476': 0.3387111118152333, '541650': 0.9438272923705118, '657022': 0.12290324697213861, '542908': 0.2792174010800577, '640470': 0.8394291577535655, '664350': 1.447024571787126, '669270': 0.16320425180870704, '665759': 0.09037055294898426, '665861': 0.05971968468649843, '489448': 0.12098191618647511, '641525': 0.6901430321436858, '665482': 0.19957737894114924, '669373': 0.7603319289622088, '666160': 1.0584755508863264, '595234': 0.15748858789123976, '542866': 0.15539601090031016, '608723': 0.5502730103547542, '665561': 0.17366145019358567, '664682': 0.14920660173791778, '607333': 0.2218938881453536, '656713': 1.0775689923111376, '521655': 0.8324980879821112, '547001': 0.013042012212469525, '664702': 0.8028747052372877, '622492': 0.15565301345888188, '660821': 0.2631617322731275, '595909': 1.0315754620580746, '545337': 0.14741533073526306, '596295': 1.0218012727251369, '641482': 0.7578197072304022, '643393': 1.826764402480102, '664247': 0.6761935385835509, '657770': 0.08246094166982001, '641401': 0.2050366539023548, '647304': 1.0845394334444294, '544925': 0.17828523714111447, '621107': 1.8001979793639458, '668804': 1.624029957243345, '650382': 0.655163189807119, '571735': 0.7261718575297039, '592127': 0.12986442155686181, '607473': 0.2658767243252086, '641312': 0.6281266140599214, '664059': 0.3933902747980194, '657193': 0.32626168454557636, '594987': 0.5739831229613435, '605521': 0.19950997270431173, '606336': 0.061167373285513894, '450729': 0.5682341839529254, '646240': 5.265917889201437, '620443': 0.35604376825312617, '607572': 0.2698146863412178, '607208': 3.4716200021174295, '621002': 0.768426263339533, '448306': 1.1663086516625658, '640457': 2.5072176507657944, '608334': 0.8900558379606479, '449181': 0.3390474431396644, '625643': 1.540608503912256, '622441': 0.1079030704044861, '571448': 9.919112117132066, '450172': 0.7183900224858014, '607387': 0.22261963128627144, '594011': 0.6433871408063797, '456665': 2.4752129364208337, '425844': 7.225253856577795, '448801': 4.279324094250847, '592314': 2.601948634972986, '501822': 0.1663431806035791, '592696': 2.797600469286298, '424144': 1.1474221191428482, '518649': 0.09764512438956387, '456488': 1.9643597923983307, '572122': 2.373619406479977, '572070': 1.7757206311242053, '605480': 1.822202006412191, '605182': 3.2938151124245567, '641427': 0.10362832119454621, '572821': 2.8387305369710347, '642423': 0.6274663885662196, '474568': 1.0573491322125341, '543333': 4.028988724167161, '572039': 1.8967925728056574, '519346': 2.7935256239111435, '518595': 2.7201567316429682, '608324': 7.714159745977281, '543294': 3.451954837970053, '468504': 1.8823160055747283, '460075': 4.748560454910741, '571506': 1.3783124475405764, '572041': 3.33641134400081, '514888': 8.789616841675091, '502085': 1.5066276823781048, '541645': 1.9352167674887697, '543135': 3.685839931442871, '519333': 0.46408532793592844, '608717': 1.0506910957356876, '543045': 0.38858571486342997, '641343': 1.4107435726100563, '601713': 1.7922008820407178, '573186': 4.4074229274162, '453343': 1.7617364741512305, '608336': 4.387576682155847, '607231': 1.127561981469075, '621242': 2.886749723456186, '488984': 0.6854517491413757, '446653': 0.35068737490490737, '543760': 2.720752263389457, '435522': 2.098318579789731, '623451': 0.13736209354862436, '493603': 2.914750517129909, '425794': 4.036091508593255, '623182': 0.2681077443843093, '656427': 3.7724902998755447, '659275': 1.526350666445877, '460086': 2.271817240547602, '547172': 0.7508631764128227, '608337': 3.8316777154767023, '517008': 1.1216661022557737, '605260': 0.1898469920041304, '408236': 2.007490436168886, '595881': 1.171907152524515, '608384': 1.0832521125456231, '456030': 3.6137702855132723, '608671': 0.7250558290882502, '553993': 3.3594904868616724, '446359': 1.124026819436332, '517414': 0.1848946019168984, '608652': 0.6224478148535518, '573185': 0.8286663970884277, '543510': 2.5341208301493325, '621389': 0.689187450528985, '518542': 0.27685262833242286, '605397': 2.5449483794867773, '456124': 1.659844126373503, '620446': 0.5051906774541723, '650490': 1.53726699931125, '592663': 6.294535131668954, '553988': 0.3728876558747225, '668678': 1.2377794778626325, '519381': 1.4199471123483134, '573046': 0.2935701634757897, '571970': 4.779802915427702, '518813': 1.525244071754688, '523253': 1.1532813410185196, '501992': 0.32377530988218695, '518568': 0.40288936271900405, '542914': 0.9155548638865917, '669222': 1.7358047082544779, '518468': 0.09562803930287982, '641355': 12.343830996312256, '622666': 1.9850658266963384, '453329': 1.558222523101848, '457429': 0.2177347518350612, '572140': 2.787932295127211, '502171': 1.8611781047656295, '621433': 1.3467736761181675, '544836': 0.17933707510389743, '519048': 2.624572514347888, '444468': 0.8450968005086928, '595375': 0.582674217159093, '643376': 1.451771654286327, '622046': 0.5779050101019974, '542921': 1.4926703034626714, '460283': 0.6854294915107904, '621199': 0.5154561902066604, '641438': 0.33619170264835563, '605233': 1.147166863679818, '595978': 1.206709333160979, '607074': 1.2698051706001787, '595014': 1.8539050792147462, '607192': 3.0260315093500174, '493596': 0.9478964280777402, '594824': 0.8644599225793911, '650828': 0.31089279938645925, '621563': 1.216551432718463, '608700': 1.2207464484562793, '622786': 0.16608054616566967, '405395': 12.723929091192577, '656555': 4.160666529750726, '533167': 1.481944410395557, '500874': 2.8363549871374114, '543105': 1.247065915235229, '519293': 1.7445599127505453, '571927': 2.6624295016860224, '600869': 1.4519547973746028, '622608': 0.9650793077607113, '606983': 0.20051917608252015, '476704': 2.239079536050473, '543101': 1.575866165867196, '488818': 0.5873482755075257, '592254': 0.5628152378491103, '444482': 1.540531373601595, '608365': 3.042190529350778, '455976': 3.8064299515128965, '570267': 2.0131806238963734, '621058': 0.2164511799256647, '622663': 4.389868413113709, '641856': 1.2994933967123163, '456696': 0.5209170494877908, '453562': 5.542860836794108, '407845': 1.8260628908929344, '642336': 1.408850551847724, '446308': 1.4434591385500266, '593160': 3.1197613422651953, '592135': 0.7222136531053985, '431151': 2.459241438050155, '518692': 7.705845538225997, '489149': 1.242493668807118, '545350': 2.001016615934676, '544725': 1.1051691573847822, '642701': 0.6484706183791191, '544369': 7.7337172839158805, '594809': 2.2787603865709074, '605137': 3.809201843226906, '543867': 0.20807312756324667, '641816': 1.2086352411063646, '518618': 2.5201694273524184, '542881': 1.2016913080742224, '592626': 5.671334924987784, '656756': 1.7712224917815937, '588751': 1.5694554497071838, '605113': 1.3645446279550526, '434378': 14.34708652849739, '607223': 0.38388611650494797, '519008': 0.36924669333103605, '457708': 2.7942847576381205, '621035': 2.8954372134932584, '435043': 0.5425597856365469, '502202': 0.770806096971832, '502570': 0.29733046185131984, '434628': 0.6153841704355245, '517593': 0.5297802098548449, '502028': 0.13224914731389242, '572816': 2.4125989234371503, '542432': 0.16251539851085098, '635903': 0.09953193034394832, '543257': 1.1718244597688454, '488748': 0.5204630948809139, '571679': 0.3510386258998502, '519141': 2.030921205074352, '622766': 0.17683934720623248, '641487': 1.7265577406957615, '472551': 0.40695598365253344, '592808': 0.9209900655152962, '571912': 0.7723622020931014, '605200': 2.2696411562392282, '592815': 0.4581537595019127, '434158': 3.0122615369875505, '642708': 2.599582096543292, '435062': 2.2237108787917625, '453178': 1.0318327981305504, '594056': 0.19143814357059363, '592743': 2.832253922184766, '502143': 0.5258509908339757, '502032': 0.7802933142729799, '571656': 0.4450777226984249, '573135': 0.5898418238958327, '622075': 0.3079237027456425, '457803': 3.7866813467504805, '443558': 6.214633003310763, '628711': 3.0272737983802243, '664023': 3.4430925495651814, '650391': 2.9929154672129634, '456715': 3.5151348859845952, '545333': 9.425690268838608, '489267': 0.3252994712414023, '571771': 4.552027806513216, '455117': 1.808441701377218, '608371': 0.6260586225169604, '489265': 1.9958236451448266, '605204': 2.7005705254889256, '541652': 0.1642967754011079, '554340': 0.6207133573561879, '502481': 1.254447505059618, '488726': 3.8358147038504224, '545121': 0.7687958329008615, '592717': 2.1378296008583213, '453385': 1.005080087770143, '592192': 1.3635697780296459, '642547': 1.238093884659864, '640463': 0.21978850554595278, '594838': 0.6008001770259804, '571745': 2.412540929041361, '543037': 12.485750658356748, '656302': 1.1714642764987553, '457706': 0.8896403817446683, '408234': 7.955279620037041, '425492': 0.6410023247635569, '607536': 1.738856064444321, '543484': 0.6638310300764154, '621381': 0.7701872694393789, '621052': 1.4605107985455412, '547943': 8.075929263891362, '488768': 1.500994052307859, '657557': 3.2919737802887035, '519023': 0.7795144613389359, '446192': 0.10161324711406416, '596720': 0.3654658035549287, '456034': 6.167632302512466, '503285': 1.100334278532179, '605232': 0.5351952623958193, '592348': 1.5488921060986964, '467793': 3.689364405964619, '656954': 1.8949633036654625, '605400': 3.8753825945494347, '475247': 0.7413282549830363, '459964': 1.2339238290066923, '595001': 0.34806621600538956, '518735': 4.522242280198373, '468396': 0.15596991931596163, '624431': 0.48051530581530394, '518774': 3.9366403990542955, '532077': 2.221513465986327, '621512': 0.7005293591809351, '573109': 0.48008560989572635, '502706': 1.9568844764802877, '622682': 0.9157044175233902, '448855': 0.9318296926902114, '450314': 3.499860136469349, '666971': 2.8167057512588416, '624586': 0.3213714363975711, '542436': 0.35527968431575835, '608597': 1.2455103260736033, '605512': 0.8606438030809362, '592570': 0.09577501359652169, '572096': 1.5296805207270008, '641820': 2.8029861842009214, '453344': 1.1853665432210334, '461833': 1.051920195760771, '592170': 0.3928726657888359, '429665': 5.362271326494548, '518748': 0.44140776011032534, '519443': 1.0804858850397199, '543548': 0.8631104137072172, '519076': 1.1114746875534791, '595885': 2.7360061488580922, '471083': 0.4171428221351507, '444876': 0.9796398837144975, '502226': 0.25871495983854076, '592178': 8.76193931178312, '453286': 9.603986646205298, '400284': 2.325774785521127, '433586': 0.2524630573346107, '596847': 2.3757203050778095, '446372': 5.40512781092603, '502210': 2.4000228847584064, '519222': 2.222926834013271, '543521': 1.367002765544778, '501789': 1.1799120474419658, '640455': 1.7778602416918858, '656805': 0.2991321985051072, '450212': 0.6719129485674799, '451594': 2.9701097912205117, '474463': 1.6052308531428023, '461872': 0.8441957936742389, '475115': 1.0463614036091111, '656794': 1.9665156387624203, '501957': 0.4367105747400124, '606930': 0.5128835247761754, '621566': 2.1276295828660494, '434670': 2.6224213659644744, '641313': 3.920618179626645, '502026': 1.4476328431416312, '519295': 0.1609467560975025, '605177': 0.5105632210683052, '572669': 0.3824757539335101, '400121': 1.0937746151768133, '542255': 2.0847356853016628, '572191': 1.3763464619615027, '542513': 0.152803011176082, '502054': 3.910331040036446, '669221': 1.2031159882057743, '628356': 1.1369229715071978, '546991': 2.78816444961481, '592273': 1.927865897320561, '542888': 0.4186173988842851, '488771': 1.3030731384960421, '660813': 1.6353815827445883, '676801': 0.18555689459603275, '663656': 1.6661370968228129, '606478': 0.1646714796124371, '663432': 0.4806618170619142, '665620': 1.374833209218086, '669302': 0.34929015218525505, '592165': 0.2816621937500473, '641627': 0.5091157570559384, '592145': 0.5591842630132957, '594835': 2.430065297171919, '525768': 0.4156895415950615, '506702': 1.1999301097057766, '656849': 0.80537506815761, '685503': 0.26924544147571505, '665155': 0.17879739935537486, '500610': 0.2733275936564651, '592685': 0.6836096784111171, '664141': 0.37145416033073064, '663559': 0.04200366163187392, '666176': 1.2761870337939119, '523260': 3.3087278343740403, '593643': 1.356191494569485, '621500': 0.1504980001578924, '641929': 0.10834207780397831, '657757': 0.12451162191596475, '658668': 0.3628753244547431, '642239': 0.21773542582115718, '657240': 0.3531031367723719, '664238': 0.7477299128479981, '665834': 0.14278679608035674, '596144': 0.6663211086486996, '607345': 0.5544891655586721, '600474': 0.9432237110928202, '685493': 0.812756921630618, '656821': 0.11457330095798089, '596825': 0.6127597755348112, '622774': 0.21434740335383862, '593679': 0.3453062227256048, '666200': 1.8109153830314575, '607755': 0.09330473193421382, '657031': 0.4380229771648919, '621139': 0.20819008916571075, '641962': 0.15122220969808628, '666622': 0.19319815211124403, '671250': 0.2502188129848705, '656420': 0.3003035384171785, '613534': 0.44196109044373916, '593590': 0.04284233912260117, '656354': 0.2884775218960727, '571893': 0.18485728675789515, '656803': 0.3152533032704991, '623433': 0.864189042156125, '607054': 1.0782971331248132, '663554': 1.7058210931311641, '624513': 0.24754576534738207, '571035': 0.19947553492976447, '606157': 1.8078596231653272, '429719': 1.2950061473297565, '656514': 0.8290015330280316, '622505': 0.20889225625660635, '623180': 0.13140318898328548, '640458': 1.3261917324795347, '554431': 0.34929247421683823, '608335': 0.2861915240534037, '605894': 0.044450424559583625, '621249': 0.40781079938538906, '673258': 0.3186529692608299, '596056': 0.2691207578282083, '664119': 0.4548425825158578, '622168': 1.2990095949790812, '656548': 0.12965448409748892, '670097': 0.34829513507058435, '608475': 0.49290883853268513, '502304': 0.2056951847803726, '641540': 0.6822305780869432, '456387': 0.22403295970865406, '591971': 0.8300752873042642, '660688': 1.1231551862595466, '592612': 0.49220778833762896, '665750': 0.4114708443516506, '669134': 0.48980371742702095, '642201': 0.2071080106873638, '676886': 0.07754404813551746, '622864': 0.44603653974903656, '656180': 0.15641288941710013, '669160': 2.472736900525669, '596012': 0.34624195088655163, '445988': 0.6234524545405589, '605240': 0.3951850438491876, '641914': 0.6614469500317091, '664701': 0.16192069336438528, '663366': 0.16587381258721337, '553902': 1.7053595009250422, '573062': 0.04419356569428107, '657766': 0.062108020071183344, '671790': 0.20644586673025794, '673357': 2.935360293904957, '624427': 0.39432468603455023, '620454': 0.35271810715104607, '676755': 0.07645430264414226, '676103': 0.6907314257383322, '467092': 2.7311797579278414, '624407': 1.0742361181943911, '641851': 0.22885579192914776, '622088': 0.37684458218167904, '543507': 2.006507204436816, '629496': 0.2681740672947502, '547942': 1.7769035622141145, '621493': 0.5461498065581348, '643290': 0.8278673717441627, '623402': 0.17406280508063432, '670970': 0.5370960700191922, '489295': 0.11946583745364817, '516472': 0.2532092129871771, '664842': 0.09200977223372199, '607067': 0.4087963007952135, '668227': 2.346724938542795, '663770': 0.2103697118919016, '673490': 0.45889484503816125, '664068': 2.337154840856178, '592833': 0.30006971247001, '608654': 0.339203106936941, '596074': 0.30833715214167173, '527043': 0.25618181355949243, '476594': 0.4337274457075253, '606944': 0.2921034735596853, '592094': 0.34335926615871604, '675916': 0.633506713195646, '641394': 0.047719891414673296, '621219': 1.180814417921545, '666135': 0.7429232884385796, '621141': 0.5887841255557005, '664285': 1.189160708083738, '450665': 0.2901533118885723, '571476': 0.12292603187348404, '626929': 0.43013930004881107, '641682': 0.15614596547883544, '650668': 0.21463487084806698, '612434': 0.7712393898101978, '663765': 0.3377629627643061, '672715': 0.6945969343547831, '592848': 0.11211441899327093, '641447': 0.20416895315420258, '656529': 0.2048357491843567, '605196': 0.5987576757228538, '624415': 2.981733602817145, '666201': 0.4091367405581162, '665489': 8.575887021917156, '595345': 0.1086731314723891, '643532': 0.2681023807599168, '664058': 1.524523183447555, '572193': 0.7279169534677422, '656266': 0.504359379159006, '518445': 0.10083101180677422, '665926': 1.0622135937311115, '571871': 0.1900767153594008, '642221': 0.5208294971636169, '582473': 0.05199197958491714, '650347': 0.11531766328061141, '641726': 0.2748495542139235, '640459': 0.09117830307762846, '607216': 0.07520591921862123, '656557': 0.573901576135451, '620439': 0.8527478010727135, '657656': 2.2332366083192463, '650633': 0.6913220234684193, '661403': 0.6834847512685226, '518858': 0.5015649615676816, '518715': 0.5638136816247886, '596816': 0.08775589863361231, '657141': 1.0843824313583, '676596': 0.30112061397025314, '621086': 0.09589939973042277, '605488': 0.43565923027880926, '605353': 0.29413049240599803, '660853': 0.4122487169137875, '642028': 0.1927350290616813, '664034': 0.8721511589421226, '669256': 0.9791278080723655, '642840': 0.338623117313966, '572086': 0.1625790593225245, '643354': 0.23837731229540346, '503569': 0.4615379780912177, '594840': 0.29113460832115745, '622161': 0.1053247783186517, '608422': 0.043131452976207865, '676684': 0.06041500592336422, '664854': 0.526543172901922, '641656': 0.2933070807635623, '641755': 0.29469178294512954, '488786': 0.17012504851776786, '592622': 0.3243150963929956, '542960': 1.0948428989375747, '661269': 0.16848401042622116, '407822': 0.37764136128674264, '656186': 0.3826267440949735, '664334': 0.06564687712044817, '641943': 0.2474009682446571, '623214': 0.3067919148284875, '656568': 0.09741657944707119, '528748': 0.07715647021354913, '662139': 0.7377882574066723, '664074': 0.1943410240611736, '669289': 0.5358408194114183, '519294': 0.2143741111038542, '543508': 0.16152232108375916, '622250': 0.3153126498277282, '665093': 0.13776737292930924, '593619': 0.1523471870695645, '677651': 0.2734050298532871, '518875': 0.5608557327993239, '607200': 0.794556386924311, '642232': 1.279667806893976, '656582': 0.4516347056070771, '656257': 0.22950282711849682, '664731': 0.10875799668349452, '599683': 0.15077381427214595, '621368': 0.26556481270737875, '572033': 0.6757320983175541, '519166': 0.2767032199037154, '622103': 0.2551834164574591, '641927': 0.023420406336317475, '641743': 0.6176609076445725, '657624': 0.903838662815842, '543391': 0.29654248163061836, '664157': 0.2843627031491424, '527049': 0.44044624046957676, '664728': 0.25090809248495843, '680911': 0.09614434178404237, '663898': 1.0579826734658564, '656061': 0.5650042717081782, '641786': 1.6547771150443444, '592660': 0.6303495936030852, '596119': 1.281302552504723, '446899': 0.4434315113619913, '641386': 0.20021821801801715, '592238': 0.06629624782068007, '641672': 0.34649303487603017, '576788': 0.13128254346433185, '621550': 0.6126499996988959, '456068': 0.3293681569823796, '553882': 1.3220962365554318, '572955': 0.40885704983484805, '622100': 0.26746280851400683, '656458': 0.027018095617130044, '643565': 2.3535056530169176, '621385': 0.19043107149767502, '669424': 0.18342396289489143, '593528': 0.6146270321429497, '502522': 0.3397839527434302, '607560': 0.5243887147708136, '641878': 0.3917010264910153, '667463': 0.05873398971561356, '596105': 1.0526577731626257, '573127': 0.35365831037639156, '592288': 0.17351053617122378, '657006': 0.17427978799231886, '643446': 3.7776026357354535, '543031': 0.09624618365573294, '621044': 0.20911945489731715, '643410': 0.3861783155439724, '516969': 0.9670180671753393, '571882': 0.4339870251741926, '545064': 0.27664474786600557, '622253': 0.4976240600164444, '595411': 0.25194714489021913, '621237': 0.9648042590665176, '607732': 0.660700935484593, '660896': 0.20708957998713357, '448178': 0.11924515118477239, '605276': 0.7194601710021467, '656492': 0.11480063863528793, '668687': 0.13332153554485227, '664299': 0.5623290426803716, '425784': 0.5301793988490315, '621142': 0.561007996357681, '665506': 1.346212808527515, '657061': 0.9635140474400706, '666198': 1.084977334252395, '642100': 0.17169585355631475, '623470': 0.17024056774348081, '605309': 0.9180381102149419, '656222': 0.6673955971056699, '458690': 0.18969114798321232, '595453': 0.3768077107867274, '661255': 0.3431225599923861, '519151': 1.0228101993780865, '669952': 0.47202665601987737, '571473': 0.10343563274055688, '663474': 0.7815076095377649, '641853': 0.1966669224554601, '622490': 0.10805030060101756, '662253': 0.3853612819330564, '621350': 0.10882857370492066, '474319': 0.21594821931757222, '580792': 0.4188537739127739, '629498': 0.11203643929551815, '669145': 0.29059613891060343, '664076': 0.3750892895069457, '623439': 0.08272752117883116, '660494': 0.31333168380015064, '608686': 0.486265861483872, '657041': 0.8245759424373872, '625510': 0.24447413397150086, '621345': 0.7182462293223975, '623993': 1.2042586358665583, '663538': 1.770420798091095, '663738': 0.24574449700978224, '656716': 0.82612320403412, '657209': 0.13752218190405158, '606213': 1.0048078253526385, '543359': 0.5042359951303458, '446868': 0.14406009237345538, '665120': 0.7479602937497578, '663993': 0.9381018004926569, '657044': 0.18612178675695434, '543194': 0.1715072114795827, '673451': 1.114513586039309, '642545': 0.9557397234524956, '642067': 0.21804077160508809, '622503': 0.14277610260296034, '592390': 0.6533235993747745, '457754': 0.1662367328729951, '642727': 0.41313721187216546, '607030': 0.19023343797745743, '650738': 0.12102202098995991, '472528': 0.7908224515725772, '656798': 0.34279539675054427, '656605': 0.9553693019480359, '593495': 0.24366362858478438, '650619': 0.3235514853756544, '664192': 1.589023215478988, '667674': 0.07258403217154841, '657681': 0.06383505308543028, '622382': 0.17909327652099408, '641571': 0.39488542170921487, '656537': 0.2759301186442634, '670541': 3.2433722277531074, '621294': 0.48279812673846306, '572044': 0.20775317305827665, '541600': 0.21896942741554196, '680704': 0.05156700706037207, '605301': 0.22559696001470164, '670912': 0.19352540801613083, '608331': 3.4272775235256834, '642772': 0.4184094171295526, '448167': 0.12040331528956089, '657566': 0.13332295365755126, '660546': 0.3143908064460764, '621076': 0.3307328488038834, '606303': 0.21711778865500012, '664353': 0.8819097877949578, '669459': 0.054250494590971796, '519240': 0.346947235361976, '663465': 0.7095264151888652, '642758': 0.32989168900201987, '605525': 0.27504401647358173, '573131': 0.6401280940381099, '664199': 0.4798525783997767, '663399': 0.125201937689924, '656775': 0.7566033098748627}, 'teamId': {'119': 3.382704896153294, '145': 1.4268195020216592, '147': 4.371858039002097, '121': 2.3862369167477167, '113': 1.58273400624382, '117': 2.4604764022881915, '118': 1.0017463255499728, '135': 1.518564009598897, '138': 2.0440555172852273, '158': 1.7222267925370893, '136': 1.3331258686576937, '115': 1.1542005012523084, '112': 2.6260671599868237, '114': 1.5796967263100394, '109': 1.1234739761123675, '108': 2.249598671278583, '133': 1.2556176757073605, '144': 2.2669614535871543, '140': 1.2196565104793033, '143': 1.7349962187068384, '137': 1.668901571409611, '120': 1.854969499584884, '141': 1.7603683802726466, '134': 1.067641677579018, '146': 0.963707902012593, '116': 1.1689877107923634, '110': 1.1343531319215454, '111': 2.772532461093106, '142': 1.5342108803542258, '139': 1.3630764486008549}, 'status': {'Active': 2.0949996286824235, 'Injured 60-Day': 0.4734157841342181, 'Reassigned to Minors': 0.37493374481437514, 'Injured 10-Day': 1.0990591598504376, 'Injured 7-Day': 0.9557835561331173, 'Paternity List': 2.8424569133817243, 'Suspended # days': 1.8056312168994482, 'Family Medical Emergency': 1.3917213742841688, 'Bereavement List': 1.5626987750692911, 'Reserve List (Minors)': 10.960295638565192, 'Deceased': 100.0, '60-day IL': 0.6368929393567545, '10-day IL': 1.0721263686777558, 'Reassigned': 0.3067804087642798, 'Paternity': 0.9183441201733071, 'Suspended': 4.488729133095335}, 'seriesDescription': {'Exhibition': 2.217268772541211, 'Spring Training': 2.457423284329911, 'Regular Season': 2.119418906477148, 'Wild Card Game': 2.5320855597974705, 'Division Series': 3.4995014058990304, 'League Championship Series': 5.3257823808784215, 'World Series': 7.4998028933767324}, 'positionName': {'Shortstop': 4.804434400312083, 'Pitcher': 2.230243471167579, 'Third Base': 4.96992726600378, 'Second Base': 4.035747987627302, 'Outfielder': 5.055941345937113, 'Pinch Hitter': 2.451944403312413, 'Designated Hitter': 6.510754129548734, 'Catcher': 2.591907846122347, 'First Base': 5.388117205112038, 'Pinch Runner': 1.6787567098239407}, 'positionType': {'Infielder': 4.800548075466637, 'Pitcher': 2.230643957254252, 'Outfielder': 5.055681769634792, 'Hitter': 4.309392602908981, 'Catcher': 2.591907846122347, 'Runner': 1.6737629701284678}, 'awardId': {'CSAS': 1.413789786648357, 'ALPOW': 27.176497436951195, 'NLPOW': 26.27564001819651, 'TEXPOWH': 0.04119749585357925, 'INTPOWH': 0.4463003039388243, 'EASPOWH': 1.0393168649371085, 'FSLPOWP': 0.42972714129942685, 'SOUPOWP': 0.02848958408842106, 'EASPOWP': 0.17945989748875887, 'INTPOWP': 1.3206865022791026, 'SALPOWH': 0.011032565450710416, 'PCLPOWH': 0.5779294892804856, 'SALPOWP': 0.014704457011048468, 'TEXPOWP': 0.44740002697752146, 'SOUPOWH': 0.1087256411618834, 'PCLPOWP': 0.1925771082644902, 'LOUGEHRIG': 3.1316483848232073, 'CARPOWP': 0.06413165298809925, 'CALPOWP': 0.032738653554093326, 'CALPOWH': 0.07896904273152362, 'MIDPOWP': 0.0023113533207003755, 'FSLPOWH': 0.1359984203641526, 'ALRRELMON': 3.813384227040111, 'NLPITOM': 17.00766020553004, 'ALROM': 13.774575150658276, 'SOUPOM': 0.10477908792931004, 'SALPOM': 0.03438957876464026, 'NLRRELMON': 8.656036562052897, 'PCLPOM': 0.7778322284666589, 'EASPOM': 1.2840745047313453, 'INTPOM': 2.068790752234755, 'NLPOM': 20.777397364516652, 'ALPITOM': 13.343731898448624, 'MIDPOM': 0.050567487763276374, 'NLROM': 9.499590087352027, 'ALPOM': 13.5170128861421, 'FSLPOM': 0.13373688483172155, 'TEXPOM': 0.09257398994600903, 'CARPOWH': 0.06852894982947914, 'MIDPOWH': 0.0477105226272007, 'CARPOM': 0.6842083437297581, 'FSLMSAS': 0.036476015232037313, 'SLMSAS': 0.04419376489934186, 'CALMSAS': 0.013677261444521732, 'SALMSAS': 0.031122858571111616, 'CARMSAS': 0.04402942038343359, 'MWLMSAS': 0.014483319229818744, 'NYPPOWP': 0.0015518083772818126, 'TLMSAS': 0.03918440337168416, 'TLTOPS': 0.017127650591002998, 'CALPOM': 0.023145070280050167, 'NYPPOWH': 0.00857632933104625, 'PCLMSAS': 0.8396134977886066, 'ILMSAS': 0.2959036536056969, 'ELMSAS': 0.1489880583934689, 'ILTOPS': 0.1741887554308469, 'PCLTOPS': 0.19324435941056162, 'FUTURES': 0.28459546165997934, 'NORPOWH': 0.0412292744750935, 'HRDERBY': 21.85646137154222, 'HRDERBYWIN': 91.87181212943011, 'NLAS': 12.402074630194424, 'ALAS': 13.385678793473232, 'ASMVP': 64.37404693476385, 'PITHEARTHUSTLE': 8.906387480088934, 'BOSHEARTHUSTLE': 17.198342468114298, 'NYNHEARTHUSTLE': 7.398222940184364, 'DETHEARTHUSTLE': 0.41500198415193956, 'KCAHEARTHUSTLE': 9.84737278001751, 'OAKHEARTHUSTLE': 2.0103873107268018, 'CLEHEARTHUSTLE': 5.329537903543635, 'NYAHEARTHUSTLE': 9.392993221788505, 'SEAHEARTHUSTLE': 1.2177204521359384, 'SFNHEARTHUSTLE': 4.604025452138757, 'PHIHEARTHUSTLE': 1.3955184170484474, 'COLHEARTHUSTLE': 2.016800374117046, 'WASHEARTHUSTLE': 1.7818500852768648, 'ANAHEARTHUSTLE': 8.87035908481257, 'TORHEARTHUSTLE': 2.7176364868627494, 'ATLHEARTHUSTLE': 1.6054512671228744, 'FLAHEARTHUSTLE': 5.4239741602306495, 'SDGHEARTHUSTLE': 3.50776828521697, 'TEXHEARTHUSTLE': 2.0156106982284046, 'BALHEARTHUSTLE': 6.365785719154328, 'CHAHEARTHUSTLE': 1.2781835048724735, 'CINHEARTHUSTLE': 1.9903973729568492, 'MINHEARTHUSTLE': 2.2897506559041854, 'HOUHEARTHUSTLE': 26.045620511236443, 'LANHEARTHUSTLE': 7.187953051273574, 'ARIHEARTHUSTLE': 4.644974131904474, 'STLHEARTHUSTLE': 1.9125093836915061, 'MILHEARTHUSTLE': 4.896797979942024, 'CHNHEARTHUSTLE': 9.4109281131017, 'TBAHEARTHUSTLE': 6.259377401183111, 'NORPOM': 0.20197173710021377, 'NORMSAS': 0.04897860520081794, 'PIOLGAS': 0.009145783793671002, 'NYPMSAS': 0.0020086774867425, 'MWLPSAS': 0.20153589348500903, 'PIOPOWP': 0.03799392097264425, 'CALPSAS': 0.015923242594541177, 'ELPSAS': 0.12125859872354447, 'APPPSAS': 0.004415205969358249, 'AFLHOF': 7.074947975990816, 'APPPOWH': 0.24573770154025257, 'PCLPSAS': 2.0992789971042742, 'ILPSAS': 0.661752581500568, 'SALPSAS': 0.00730686945710825, 'FSLPSAS': 0.049076134906877344, 'TLPSAS': 1.0576403942776056, 'PCLMVP': 0.27692844364368946, 'NWLPSAS': 0.11511229939198263, 'AZLPSAS': 0.0, 'ELROY': 0.9260476071910377, 'PCLPOY': 0.25339304461688417, 'ELPOY': 0.008487171250781002, 'PCLROY': 0.47216506065960906, 'PIPELINEHOY': 4.00162409688824, 'PIPELINEPOY': 0.36866117785747554, 'USAMLPOY': 4.758796213661463, 'CARPSAS': 0.017340964179555315, 'BAAAAAS': 1.2206879332000977, 'BAMILAS': 1.1943330055287653, 'BAAAXAS': 1.1264380720319855, 'SLPSAS': 0.033755279801950526, 'BASSAS': 0.14237189050757368, 'BAHAXXAS': 0.08759035412734997, 'BAROAS': 0.0, 'ELPMVP': 0.117201162411497, 'BALAXXAS': 0.0, 'SLCSMVP': 0.0945900552191085, 'MILBHRAAA': 0.36832180521385904, 'MILBHR': 0.008861836993398499, 'MILBHRAAX': 0.0051574835278745, 'MILBORGAS': 0.25867790993416523, 'LOUBROCK': 4.09399133083367, 'BAMLART': 2.655660523599976, 'OAKCATFISH': 15.621537592034674, 'ARIMVP': 0.8897116036445374, 'BALMVA': 0.6523729443489303, 'CINMVP': 2.067206322928825, 'FLACONINE': 0.1917974959646029, 'TEXWORKINGMAN': 0.1521326772728596, 'HOUROY': 0.5982581982481832, 'SEAMVP': 0.1722950940483102, 'FLAMVP': 0.15218891429470038, 'CLEMANOY': 7.3162558431114615, 'SDGMVP': 0.2712417900314307, 'HOUKILE': 0.7575701974584513, 'ARIPOY': 0.634840654204987, 'DETROY': 0.18570855009280202, 'BOSPOY': 12.663399892222532, 'HOUPOY': 11.010224269478124, 'SEAUNSUNG': 0.13524268230244613, 'CLEGOODGUY': 0.6477282852697732, 'HOUMVP': 19.158868734467312, 'TEXMVP': 0.20913819173345768, 'TEXPOY': 0.20422563356489942, 'BOSROY': 3.5437232342669254, 'BOSMVP': 9.729157194101791, 'TEXROY': 0.060170078607582, 'BOSMANOY': 0.456138696065125, 'BOSSPIRIT': 0.7403524162551035, 'FLAGOODGUY': 0.121630900602128, 'ARIGOODGUY': 0.2211638984393012, 'TEXGOODGUY': 0.305532922595145, 'FLAROY': 0.12292143641480877, 'CINPOY': 1.6034637643799272, 'SDGPOY': 0.312032098597896, 'ARIROY': 0.3353975264228274, 'BAPOY': 8.922648199638282, 'BAMLPOY': 12.159659149536221, 'MILBGG': 0.0227167069415435, 'BAMLROY': 19.707307024064836, 'AFLPOWH': 0.22453498777599779, 'MINIMPROVED': 0.15441852110473553, 'MINSIEBERT': 0.6840343310471827, 'ALCSMVP': 60.35400444069899, 'MINMVP': 0.24935985249460252, 'MINROOK': 0.2928741970124745, 'MINPOY': 0.1673419252313335, 'NLCSMVP': 59.75072639506609, 'AFLPOWP': 0.020188344091786003, 'SFNWMA': 5.53442461299985, 'MILNEWCOMER': 5.281782474273814, 'MILGOODGUY': 1.1570953567079092, 'MILUNSUNG': 0.9465224016727577, 'MILPOY': 1.0780144742788664, 'MLBRC': 11.789214387019845, 'NLHAA': 13.933113880710527, 'ALHAA': 15.5531411667244, 'ALREL': 2.2293391204922535, 'NLREL': 1.7534689862237551, 'WSCHAMP': 14.261002412519868, 'WSMVP': 79.88637368476931, 'AFLRS': 1.157685246098143, 'NLGG': 18.47497140305561, 'ALGG': 13.80450527696022, 'MILHARRISON': 6.0978437384770485, 'BAMMILBSCSP': 0.09825276395090723, 'BAMMILBSCOFF': 1.0244692135329276, 'BAMMILBFCSP': 0.06544684975538013, 'BAMMILBSCBRK': 0.38683139195516153, 'BAMMILBFCBRK': 0.5846379927487235, 'BAMMILBFCOFF': 0.2617796753750205, 'BAMMILBSCRP': 0.0021616947686985003, 'WDPOY': 12.68894025781956, 'WMLBDPOY': 19.083956621715515, 'ALSS': 23.63471545162515, 'NLSS': 25.225803158981858, 'HEARTANDHUSTLE': 12.912337453616505, 'MLBPCALOR': 6.910062994544954, 'MLBPCNLOR': 6.944634394507531, 'MLBPCALPIT': 7.5196521484550845, 'MLBPCALOP': 9.325210926692787, 'MLBPCNLCOM': 5.068351133102865, 'MLBPCNLPIT': 3.838980936629659, 'BABERUTH': 17.284700254984184, 'ALPG': 33.42425232317987, 'MLBPCNLOP': 12.032937890987144, 'NLPG': 42.86499357889655, 'MLBPCMOY': 2.4910434626335394, 'MLBPCPOY': 7.117293470422436, 'MLBPCALCOM': 3.6342993253467504, 'NLROY': 59.99477077719751, 'ALROY': 70.8657703567912, 'WARRENSPAHN': 42.858690167484774, 'NLCY': 70.7877034370738, 'ALCY': 43.64246833293699, 'NLMVP': 65.24154083114907, 'ALMVP': 79.86562791443212, 'AFLSPORT': 0.5537360790830101, 'AFLMVP': 8.178582294344483, 'ALCPOY': 11.657269198277433, 'NLCPOY': 18.290758969932103, 'DHOY': 8.561568673243846, 'HUTCH': 0.5696831557814007, 'OAKSERVICE': 0.2958518262847838, 'CLETOMAHAWK': 1.9603163851276837, 'TORMIP': 0.15429428337790652, 'TORROY': 1.0275894514051116, 'TORMVP': 1.2597718342458555, 'TORPOY': 6.93437080068439, 'AFLPSAS': 0.12917200529636716, 'TONYCONIGLIARO': 3.1760479414640077, 'DWLROY': 0.6294583617086337, 'PWLROY': 0.02720348204570175, 'CSMVP': 8.914168801341793, 'MLBPLAYOW': 12.238690679319664, 'RETIREDUNI_140': 22.259193538814692, 'RETIREDUNI_142': 17.77295643932962, 'MEXMSAS': 0.0568521150033235, 'SLTOPS': 1.2995437111156054, 'ILROY': 13.582406582391577, 'AAACMVP': 0.016544421772459, 'SDGCHAIRMAN': 0.06966489198252575, 'BOSGOODGUY': 0.3320166504143708, 'MINPOHLAD': 1.161602384230651, 'MINGOODGUY': 0.21641274616492728, 'MILMVP': 15.244491701145991, 'STLKILE': 1.9910184119712326, 'CINGOODGUY': 1.9020511034634664, 'MLBSECOND': 3.062592976942929, 'MLBAFIRST': 14.111473840050884, 'MEXPOY': 0.03876245497794726, 'DWLMVP': 0.18421811714047212, 'DWLPOY': 0.0220171624282905, 'VWLRLOY': 0.03908839852346388, 'PWLPSAS': 0.1659698381392969, 'MLBLEGEND': 8.833490280749821, 'VWLMVP': 0.11014893160811574}}


In [ ]:
# tt_dict_path = '/kaggle/input/twitter-encoded-dict/'
# ptt_dict = json.load(open(tt_dict_path+"ptt_dict.txt"))
# ttt_dict = json.load(open(tt_dict_path+"ttt_dict.txt"))
# twitter_info = pd.read_csv("../input/twitter-info-csv/twitter_info.csv")
# ptt_dict = pd.Series(twitter_info['playerId(target)'].values,index=twitter_info.player_twitter).to_dict()
# ttt_dict = pd.Series(twitter_info['playerId(target)'].values,index=twitter_info.team_twitter).to_dict()

ptt_dict = {-0.2156475279878565: 0.1647886026860004, -0.1839866339458649: 0.8096977214792735, -0.1668713858383491: 0.4156895415950615, -0.2068661247658759: 0.5370960700191922, -0.1712312213571707: 1.0845394334444294, 0.0548243211629715: 1.7757206311242053, -0.0812642792086013: 0.8286663970884277, -0.200878102885158: 0.3302444830215153, -0.1702898055460269: 0.2655648127073787, -0.1564309301623036: 1.1603439405374842, -0.1711540561267491: 1.037740777845348, -0.1233964950188071: 0.7095264151888652, -0.1763704257032508: 0.3728876558747225, 0.7171875095330503: 4.069933009946518, -0.1358200971166883: 1.747474497268857, 0.0008935416212998: 2.981440868640378, 1.9365447641324829: 7.714159745977281, -0.2146212304232489: 0.7230599371611379, -0.1978918084678412: 0.5912139107370784, 0.1410333165900079: 4.279324094250846, -0.21048517407265: 0.043209635054345, -0.1300327048350666: 0.8095270067723545, -0.1260355458992266: 1.7195345632021446, 1.707093951473789: 1.956884476480288, -0.0450737861408605: 1.1114746875534791, -0.1057796729135507: 1.5296805207270008, -0.1546947124778171: 0.4337274457075253, 7.303386539956309: 6.3372286091759005, 1.018656631744244: 2.4000228847584064, -0.2143974512550262: 1.894195304122335, -0.2052610879731061: 0.3510386258998502, -0.1402493813428894: 1.516467651661514, 0.1537038474252383: 1.4003032251101977, -0.2032547919821439: 0.2802009853250045, -0.1639082409901588: 0.6684632518631353, -0.2072365178718997: 0.2282842138404648, 0.0384035601292503: 2.0599983471547048, -0.1951601593109158: 0.5686324884578938, -0.1957929142003731: 0.5056956980892027, -0.2009784176847061: 0.7168771149646599, -0.2049910096666304: 0.6145700080325086, -0.1269769617103704: 1.1751402087942635, -0.1952296080182952: 0.1943410240611736, 0.0245524012685691: 1.9686093058910104, -0.2155086305730976: 1.114513586039309, -0.2047363644062391: 0.3750892895069457, -0.1462991354079446: 0.9655647959106276, -0.1663312292253977: 0.9875962327534396, -0.2095051756462954: 0.3431225599923861, -0.1732606669172594: 0.5221789538434557, -0.1453577195968008: 1.6369076349878708, -0.198879523417238: 3.2433722277531074, 0.3689022420250584: 1.6041238830239974, -0.1506589709267662: 1.5694554497071838, -0.201071015961212: 0.1839367417713392, -0.1928606354443514: 0.215465923850253, -0.1846039557892378: 0.1658738125872133, -0.198833224278985: 0.4380219832129086, -0.2041344756089504: 0.0658895541289436, 3.0726253020609007: 5.542860836794108, -0.2126226509553289: 0.5989616207490918, -0.1959163785690477: 0.2690703083795331, -0.1183807550414016: 3.38946448144888, -0.1835390756094195: 1.206709333160979, -0.2153697331583386: 1.4175599429873191, -0.0622739160018401: 0.1690983920323967, -0.177334991083521: 1.0526577731626257, -0.1443622881243619: 0.7222136531053985, -0.1851672619713157: 0.1485569278665054, -0.1961092916451017: 0.166727373993705, 0.7399821185995975: 2.6133971268241765, 0.5768934041034989: 3.335725985639527, 3.7560777479052088: 2.8077837717263345, -0.0504599192242897: 2.8029861842009214, -0.073447441366891: 2.7976004692862984, 0.0252854709575745: 2.7464186194161453, -0.1995740104910326: 0.0835321753884941, 0.1210398053877656: 2.9557410792432437, 9.543786406971606: 5.784123027943318, -0.2059864411390694: 0.5208294971636169, -0.194504254852332: 0.0976566677713673, 0.2535556555908175: 2.8815070623016648, -0.2080544693143688: 0.4454945279677184, 0.0743934235978949: 4.940761643585653, 0.815496013090197: 4.258651879736105, -0.0588014806328671: 2.7935256239111435, -0.1018596792081323: 2.275599808127352, -0.203162193705638: 0.3480662160053896, 0.8635931012119942: 3.3087278343740403, 0.0330714427071162: 1.9352167674887697, 0.4785617509772257: 1.2567710153054448, -0.2068043925815386: 0.4798525783997767, -0.0369868699926745: 1.9850658266963384, 0.1683189420670935: 2.6527629427243973, -0.1019445609615961: 2.269641156239228, -0.1567704571761587: 0.7346603375847064, -0.2155240636191819: 2.621783210536654, -0.1847814358192076: 1.3127389385651638, -0.1933390598729655: 0.2599440134549797, -0.211866431697197: 0.1928054214787043, -0.1713315361567188: 1.046632814122266, -0.0799601868144759: 1.2643010918181572, 0.0512438544714083: 0.8075960189459157, 0.3414545695640874: 4.033332706747947, -0.1924053605848639: 0.349290152185255, -0.2137338302734003: 0.9225670527729108, -0.1766250709636421: 1.104563750704543, 0.2857721392918447: 2.5341208301493325, -0.1306808927706082: 0.480515305815304, 0.2901165417645821: 3.7724902998755447, -0.1730754703642475: 0.5157470932292039, -0.17918695661364: 1.526350666445877, 0.7003269066859259: 1.613963018471208, -0.1995817270140747: 0.2208304844318296, -0.2148990252527667: 3.4302557874369084, -0.0357830923980972: 1.184877875813745, -0.1814401813419513: 0.3053855003940667, -0.100655901613555: 1.659844126373503, -0.1905148124395341: 0.376844582181679, -0.1327103383306969: 1.3659359783148692, -0.0806701069343548: 2.981733602817145, -0.1790171931067124: 0.7429232884385796, -0.1734844460854821: 0.7566033098748627, -0.1338832498331056: 1.078538596167633, -0.172103188460935: 0.3826267440949735, 0.4669098011835608: 4.173815409733851, -0.1769182988392443: 0.4553743780414726, -0.1821269518927038: 1.2377794778626323, -0.1583446278767598: 0.4867309098159686, -0.1532980218071857: 1.0616989617533723, -0.203686917272505: 0.3490193247402753, -0.1813167169732767: 1.238093884659864, 0.1359635609513073: 2.1875509420120074, 0.4622876038813056: 3.920618179626645, -0.1291453046852179: 0.917763751403284, 1.686313354921246: 6.369322615961912, -0.185560804646466: 1.3081967802687242, -0.0389777329375524: 1.8823160055747283, -0.0724288603253256: 1.9543650693667955, -0.161446670139709: 1.0463614036091111, -0.102129757514608: 1.7388560644443212, -0.2021513291871148: 0.6901430321436858, 0.0490600784504764: 3.9695273767612655, -0.1747422393413545: 1.580708213270606, 0.2008055040745965: 1.7445599127505451, -0.1999058209818456: 0.1896292147815264, -0.214721545222797: 0.1683017933669716, -0.043098356242067: 1.7265577406957615, 0.429530963567327: 1.8502795535573988, -0.1601811603607944: 0.4130662441145399, 1.240846196220263: 0.8857197784571558, -0.1876211162987233: 0.3706707626555748, -0.2091116329711451: 0.1872230158358056, 1.8955314441633904: 7.705845538225996, 0.1036930615889849: 2.4752129364208337, 0.013510056795235: 0.4434315113619913, -0.1908003237920941: 1.042005347041386, -0.1479196052467986: 0.3291170369778624, 0.8079261039858359: 11.511047000171752, -0.2148064269762608: 2.935360293904957, 0.2743439686664025: 2.4276757276365126, -0.0710398861777364: 1.4605107985455412, -0.2145594982389116: 0.5028831691196184, -0.1816793935562584: 0.2681023807599168, 0.0762299560819295: 3.56332650266797, -0.204257939977625: 0.2748495542139235, 5.190795444088346: 5.134414479362817, -0.014462339232603: 6.294535131668954, -0.0608695088081666: 1.6515789947217423, -0.0883171812691376: 1.8078596231653272, -0.1347783665059964: 0.6688079279563357, 0.2820373421394382: 1.4672711756540922, -0.0244629530952452: 1.3065886779359352, -0.1445706342465002: 0.6260586225169604, -0.1856148203077611: 0.2838805454019531, 6.503530344020984: 7.80484160404877, -0.1956694498316985: 0.6260319767410539, -0.2156012288496035: 1.147166863679818, -0.1180026454123357: 0.6176609076445725, -0.0874683637344998: 1.276187033793912, -0.0419794604009535: 2.5324945808870662, -0.1729365729494886: 1.084977334252395, -0.0709781539933991: 1.992273433822228, 0.1077751022782887: 5.447490538971144, -0.0729072847539397: 1.705821093131164, -0.1506821204958927: 1.377804853154436, -0.2136335154738521: 2.2237108787917625, -0.1549107751229976: 0.903838662815842, -0.1789400278762908: 0.5779050101019974, 0.0141196621155658: 2.886749723456186, -0.0009044082475239: 1.936584851245552, -0.2062102203072921: 0.641470004336876, -0.1630902895476896: 1.3561914945694853, 0.2512329821551267: 1.840289090576084, -0.0819433332363116: 0.3215510834179397, -0.2145363486697851: 0.3527181071510461, 0.0349002586681087: 3.809201843226906, 0.2171645329239804: 0.4640853279359285, 0.6097040600787728: 2.3112663320442777, -0.0568183342110314: 0.2933334752483999, -0.1637924931445263: 0.7687958329008615, 0.8808318136881846: 3.569231101273552, 5.727070645949494: 4.581361335723047, -0.1654129629833804: 0.8191229316204246, -0.2108015515173786: 4.869336544943388, -0.0904083590135636: 1.2644261758844528, -0.1688159496449739: 0.3683768708721344, -0.1027702289271075: 0.5638136816247886, -0.1572565981278149: 2.131118393596389, 0.19764172962731: 3.9103310400364464, -0.0500818095952238: 1.0874964274416383, 0.4804908817377663: 2.6624295016860224, 0.426822463979528: 2.7754844296173595, -0.1651660342460312: 0.486265861483872, -0.2003610958413331: 0.4516347056070771, -0.2112182437616554: 0.1855568945960327, 2.530462393118583: 4.125371511673067, 0.0432032374614752: 2.5449483794867773, 0.043048907000632: 1.1375718268633166, -0.1901984349948055: 1.624029957243345, -0.1740786183597286: 1.0775689923111376, -0.1814015987267405: 0.3974862779515949, 0.3699902717740033: 2.4987525002313484, -0.0653682417417472: 1.735804708254478, -0.2048984113901245: 0.2002182180180171, -0.2155549297113505: 4.748560454910741, -0.1489999184727014: 2.375720305077809, 0.083336873803761: 0.99422293763364, 0.0466139406461109: 2.787932295127211, -0.1843415940058043: 1.076047452293033, -0.1575806920955857: 0.2550324182130215, -0.0996990527563269: 1.3121837376919947, -0.1987020433872682: 0.6326713137154534, -0.1122306861767983: 0.3536831023672568, -0.207205651779731: 0.2894700140669664, -0.1710151587119902: 1.3816725043704674, -0.175668222106414: 0.7708060969718321, -0.2001836158113634: 0.5650042717081782, 6.769518893284316: 8.277249969926455, -0.01784217632507: 5.169887916665061, 0.0203854788258015: 2.177472626976539, 0.7040771368844166: 5.612897578704586, -0.1852444272017373: 0.1528852069289431, -0.2066114795054845: 0.2626797969110733, -0.1929455171978152: 0.2734353697563103, -0.2096595061071386: 0.229107668299078, -0.1742175157744875: 0.2373103893771447, -0.093757330013862: 1.5656770406121443, 0.6395978703441092: 0.9670180671753392, -0.1889715078311017: 0.4638146893009162, -0.21419682165593: 0.4706247149979792, 0.0940551243093243: 2.2787603865709074, -0.2056546306482564: 0.6842019958434932, -0.2072905335331948: 0.2533187266037453, 0.0234566549965821: 1.3081790055103342, 0.6191722338515058: 4.389868413113709, -0.2070204552267191: 0.4003046590101026, 1.4636762321087815: 2.7942847576381205, 0.5733438035041043: 1.6461505308287665, -0.0185289468758225: 2.9929154672129634, 0.3050325808050816: 3.6604944950019616, 0.1194270520719537: 1.036740797664123, -0.2091424990633137: 0.1346013875107892, 0.0837998651862907: 2.7061665972755944, 0.3399498475708657: 0.6169062845740191, -0.1320544338721131: 1.3261917324795347, -0.206634629074611: 0.2048357491843567, -0.1288675098557001: 0.6949947188316734, -0.1896351288127276: 0.7205632838401137, 14.343031613906147: 14.34708652849739, 0.062502261589923: 2.5995820965432923, -0.1626967468725393: 0.3970472960065077, -0.2127924144622565: 1.2990095949790812, -0.212676666616624: 0.3350546088046244, -0.135758364932351: 1.5710002382056245, -0.1615161188470885: 1.084385352082115, -0.0754537373578532: 2.068875867909962, 0.0154546206018599: 1.3467736761181677, -0.2085946259273202: 0.4713317693981844, -0.1605747030359447: 1.0800739934419137, 4.098166647410344: 2.274524935364299, -0.0922989071588933: 1.0390852369753425, 0.6194577452040657: 1.242493668807118, -0.1844573418514368: 1.6483611335385209, -0.191078118621612: 0.4580399863301172, -0.1839403348076119: 0.2304442396235841, -0.1865176535036941: 0.3482951350705843, 1.3675669376186512: 3.4998601364693487, 0.056367625771404: 2.087268538687101, -0.1958314968155839: 0.1373620935486243, -0.1791175079062605: 1.3566131931366034, -0.1985785790185936: 0.6213590419993967, -0.2041807747472034: 0.2484590376914787, -0.1176785514445649: 1.2016913080742224, -0.1649422550778085: 0.5591842630132957, -0.0177187119563954: 0.7308376362674999, -0.0454441792468843: 1.4476328431416312, -0.1810543551898432: 0.4453177593623016, -0.2086100589734045: 0.3172572510625284, -0.1891181217689027: 0.2197885055459528, -0.1789786104915016: 0.2551834164574591, 0.2752005027240825: 0.4107266168885861, 0.2768055395168523: 0.4414077601103253, -0.1906074107160401: 0.1315894201157377, -0.2106395045334932: 0.4114107468554285, -0.1558444744110992: 0.5997269934609536, -0.2101842296740056: 0.2464841180774274, -0.1646567437252485: 0.2441641741078269, -0.214675246084544: 0.451938031111185, -0.1486372418897197: 1.0315754620580746, -0.1288752263787423: 0.2869234462908738, -0.1970044083179925: 0.0962461836557329, -0.1997977896592553: 0.1563341125398769, -0.1787393982771946: 0.3928726657888359, -0.2155703627574348: 0.2116922626904559, -0.1758765682285524: 0.2878254085678029, -0.1851981280634843: 0.3003035384171785, -0.2003533793182909: 0.4093010431713266, -0.1770803458231297: 0.4073622140890854, -0.0967281913850944: 0.8181608285072812, -0.1939640982393806: 0.5604260480418105, -0.2100761983514154: 0.1314031889832854, -0.1643095001883512: 0.2767032199037154, -0.2084248624203926: 0.1013980452278022, -0.1563691979779663: 0.1701250485177678, 0.2658403602739397: 0.6662533929799698, -0.21334028759825: 0.1802552640312136, -0.1905456785317028: 0.5091157570559384, -0.0612013192989796: 0.3824757539335101, -0.0838878970429365: 0.2533182357025052, 0.1735661777357638: 0.4412090737614625, -0.1786622330467729: 0.2164511799256647, 0.117490204788371: 0.3597902575474649, -0.1179409132279984: 0.7936846681838658, -0.0067380996673986: 0.3390474431396644, -0.1233579124035963: 0.6949573021941661, -0.1999444035970564: 0.1565268950924603, -0.2018195186963018: 0.3361917026483556, -0.1740168861753913: 0.7724234944625538, -0.1774970380674064: 0.1172253329429488, -0.0943669353341928: 0.4779922143778592, -0.2026143205696445: 0.1036283211945462, -0.205160773173558: 0.0974165794470712, -0.1822735658305048: 0.2658767243252086, -0.1780371946803578: 0.2882299142812806, -0.2010555829151277: 0.2521967441092615, -0.2139961920568338: 0.3067919148284875, -0.2058321106782261: 0.120600659227301, -0.12144421468914: 0.842415380925716, -0.1984705476960034: 0.6673955971056699, 0.070303666385549: 0.5996490555184749, -0.1904993793934498: 0.4571899907545807, -0.2062796690146715: 0.1776991564403483, -0.1822889988765892: 0.3929939911434695, -0.1954225210943493: 0.1507738142721459, -0.2044354200075948: 0.3768077107867274, -0.2002684975648272: 0.2424457506530206, -0.2079850206069894: 0.0130420122124695, -0.209582340876717: 0.1396011555085523, -0.0244012209109079: 0.7870372199550424, -0.1740091696523491: 0.689187450528985, -0.1915488265271838: 0.7391042456616465, -0.0140610800344105: 0.1258518347786613, -0.2093894278006629: 0.5040171277497337, -0.0264692490862074: 0.3577038491510874, -0.2058784098164791: 0.2809960321522325, -0.1343230916465088: 0.7339474840535051, -0.1954996863247709: 0.5568495530832691, -0.2140347746720446: 0.1432075417904863, -0.1947974827279341: 1.408850551847724, -0.1863170239045979: 0.3123392254698545, -0.2105700558261137: 0.2041689531542025, -0.1943113417762779: 0.2861915240534037, -0.1508750335719468: 0.6757320983175541, -0.1927526041217612: 0.3462419508865516, -0.1411753641079489: 0.5833681651414264, -0.1992190504310931: 0.3722367899504179, -0.1746882236800594: 0.420741189597302, -0.1717790944931642: 0.636259008266386, -0.2079310049456942: 0.6614469500317091, -0.2116426525289743: 0.2478997598296357, -0.0570498299022963: 2.412540929041361, -0.2124451709253592: 0.6460249279468859, 0.5757590752163011: 0.7840967909200245, -0.143274258375417: 1.1818175188798532, -0.2045048687149742: 0.1561459654788354, -0.1388604071953002: 1.0843824313583, 0.0261960206765497: 2.8363549871374114, 0.1019491273814562: 1.6421337270502887, -0.1184965028870341: 0.8028747052372877, -0.1324479765472634: 0.9978967510760574, -0.1282733375814536: 1.1347142618479216, -0.2023133761710001: 0.3209244001579064, -0.1676739042347339: 0.6008001770259804, 0.2417416588132671: 1.4800600628302625, -0.0677835134539439: 1.69642493150814, -0.1692866575505458: 0.8766870298425171, -0.2084480119895191: 1.360678762327474, 1.7233063663853718: 4.88710688840583, -0.1813475830654454: 1.705923508828958, -0.1789863270145437: 1.1051691573847822, -0.192698588460466: 0.9708488189465312, -0.1487607062583943: 0.5115524101119174, 0.2706863367444176: 1.057349132212534, -0.18689576313276: 0.5461498065581348, -0.2099373009366564: 0.1456291111612312, -0.2052302218809375: 0.3792297372162038, -0.0199719366847068: 0.9155548638865916, -0.0783628665447483: 0.5943874052629845, -0.2035325868116618: 0.1512222096980862, 2.900932664372791: 18.30147916689039, 0.0788612904393069: 2.5185395036058016, -0.2046206165606067: 0.595577781564527, 0.0256249979714297: 2.271067537830664, -0.0093540009786916: 2.430065297171919, -0.0900842650457928: 1.2111513414240296, -0.21562437841873: 1.4107435726100563, -0.1887400121398368: 0.5888544362027827, -0.2033473902586499: 0.864189042156125, -0.1758688517055102: 0.2935701634757897, 0.0306561709949195: 1.2367686126543924, 0.7785878833795351: 5.2395888685573, -0.1977451945300401: 0.9468675487021048, -0.1517701502448376: 1.7922008820407178, -0.2023982579244639: 0.3084160912315156, 0.285448045324074: 5.249373532861385, -0.0726449229705061: 1.1999301097057766, -0.0037826713422505: 1.1746695878757842, 1.9725423441241692: 4.287589006931669, -0.1713161031106345: 1.3130908977680351, 0.6035231251220008: 3.451954837970053, -0.1718253936314172: 0.6585124884534646, -0.1964719682280833: 0.4639175538426052, -0.1058491216209302: 2.1801487508166684, 0.0565836884165845: 1.1195243491901137, -0.1386674941192461: 0.9898237558571998, -0.1346780517064483: 0.4339870251741926, 0.5009628173686227: 2.8735195710535946, -0.1760849143506908: 0.400230675958784, -0.2045974669914802: 0.6281266140599214, 0.8551203589117: 5.357054667943442, -0.013752419112724: 1.232071135234364, 0.0732976773259079: 1.553631363148848, 0.4831144995721015: 2.0847356853016628, 0.135377105200103: 5.981892707704848, -0.1797888454109286: 0.4983550438547355, 0.0805589255085826: 0.6854294915107904, -0.1738085400532529: 1.1216661022557737, -0.192173864893599: 0.409504310782795, -0.057319908208772: 6.3830581175109815, -0.1488764541040267: 1.180814417921545, -0.1289292420400374: 0.4255117945436122, -0.2120747778193354: 0.1450866661659035, 1.3206967766605575: 4.293772569926188, -0.1635764304993458: 0.6048870977362597, -0.2076995092544294: 0.051567007060372, -0.1979458241291363: 0.2935681617608673, -0.1415380406909305: 1.3634895011049877, -0.1560759701023641: 1.79348423310713, -0.1633140687159123: 0.5258751186643374, -0.1753441281386432: 0.8875675145091091, 0.625893325421229: 3.567138498980395, -0.2138264285499062: 0.189558261525839, -0.0867584436146208: 1.6232633171161306, 0.9745335529891604: 3.3166009508202747, 1.4523946754211403: 3.5726790625198133, -0.2144051677780684: 1.9685587061990124, 19.407393403000256: 23.08291866948413, -0.1433437070827964: 0.542520521114502, 0.7434931365837814: 9.703695308112682, 0.0259027928009475: 2.4761414195108493, -0.1490076349957435: 0.4091367405581162, -0.1227097244680546: 0.6076437330167073, 3.441621717414056: 4.4074229274162, -0.1917726056954065: 0.7883320181064699, 0.7361547231706851: 1.6852362967943897, 1.7108827642874906: 2.5502368670945827, -0.0494876373209773: 0.3506873749049073, -0.2066577786437375: 0.1622435658947023, -0.0939193769977474: 0.5628152378491103, 2.223537689116581: 9.603986646205296, -0.0230431128554874: 4.890517173482894, 0.3571731270009718: 1.1487422546726695, -0.059889510381812: 1.08048588503972, -0.0553984939712736: 2.9293900313201853, -0.1755524742607816: 0.6884990279513531, -0.1742715314357826: 0.4724377849678206, 0.0383726940370817: 3.2878414874440964, 0.5162492495151461: 4.160666529750726, -0.1034955820930707: 1.2415890506229703, -0.0738178344729148: 0.4333015933928357, 1.4964483054688449: 12.723929091192575, -0.0606920287781969: 1.931654972200048, -0.1416615050596051: 1.0579826734658564, -0.1547101455239014: 2.026060971576023, -0.18718127448532: 0.2614466818319174, -0.2009166855003688: 0.6843900975296582, -0.2018658178345548: 0.5646308937177428, -0.1407509553406299: 0.8315203476102793, -0.2061021889847018: 0.3011206139702531, -0.1743718462353307: 0.5501303859521773, 0.1257005853052316: 0.6123053095786506, -0.213818712026864: 0.1935254080161308, -0.2015108577746153: 0.3251732185524052, -0.2031081780443429: 0.4377252570899627, -0.177049479730961: 0.7410615048691706, -0.0741342119176435: 1.4363751002708032, -0.1478501565394192: 3.10769887826522, -0.1130640706653519: 1.6547771150443444, -0.1766405040097264: 1.7922122040996398, -0.1504043256663749: 0.59200077138031, -0.1906768594234195: 0.180145587044798, 1.571128815470891: 8.789616841675091, -0.2030541623830477: 0.4306837318381517, 3.3514541456663904: 4.028988724167161, 3.155176665565952: 5.290460807558066, -0.1263442068209131: 1.0629082169564013, -0.0315621542940345: 0.6854517491413757, -0.2098524191831926: 0.1619206933643852, 0.5538132836843918: 8.075929263891362, -0.1550959716760095: 0.7577107337417305, -0.1863787560889352: 0.2633861685763567, -0.1839557678536962: 1.4218009114425438, -0.1855530881234238: 0.794556386924311, -0.193076698089532: 0.3930936042854956, 0.1057302236721157: 1.5273387673277876, -0.0802997138283311: 0.4088570498348481, 0.6939067595148469: 6.214633003310762, -0.1238440533552525: 1.575866165867196, -0.1010262947195788: 1.2994933967123163, 1.1673694638127945: 7.43040091778724, -0.1527501486711922: 1.374833209218086, -0.2084557285125613: 0.1787973993553748, 0.0487359844827055: 2.337154840856178, -0.0842814397180868: 0.4028893627190041, -0.1599033655312766: 0.576139190608349, -0.1530973922080895: 0.1877374649271524, -0.2142739868863516: 3.0272737983802243, -0.1744412949427102: 0.2745574791649825, -0.1930072493821525: 0.2383773122954034, 0.3042300624086968: 2.8387305369710347, 1.0278624437335435: 9.469104228139225, -0.1510602301249587: 1.6601545423448585, 0.4100467428858674: 2.2945943042604697, 1.018317104730389: 2.098318579789731, -0.1540696741114019: 0.8210682173112321, -0.173955153991054: 0.6945969343547831, -0.2117352508054802: 0.3793085050665545, -0.2120824943423775: 0.1296544840974889, -0.1943730739606152: 0.5154561902066604, 1.000106110350886: 4.763200991781353, 0.189269302126564: 3.2681546445941403, -0.184418759236226: 0.5987576757228538, -0.193933232147212: 0.5690682928874539, -0.0993826753115982: 1.0298032538269415, -0.2006234576247666: 0.3464930348760301, -0.2013256612216034: 0.1148006386352879, -0.214436033870237: 0.0615999085233062, 1.2394803716418004: 0.92754595418609, -0.0827304185866122: 1.3278584518178411, -0.1890641061076076: 0.8245759424373872, 0.1835976576905747: 2.8869054197144632, -0.1833538790564076: 0.6588576409862622, -0.2085714763581937: 0.5093615963631989, 1.806367020411206: 2.970109791220512, -0.1705521673294604: 1.482972819370214, -0.1846888375427016: 0.6542579596869015, 0.0640918653366084: 1.808441701377218, -0.0488240163393514: 0.8310518816880614, -0.0249799601390701: 1.8611781047656293, -0.2043582547771731: 0.1094085385104847, -0.180807426452494: 0.8607015847971067, 0.0708978386597955: 3.8316777154767023, -0.1729983051338259: 0.5105632210683052, 1.2646825358975022: 13.505949455570915, 0.799561393008132: 0.4171428221351507, -0.2143511521167732: 0.9432237110928202, -0.1988640903711536: 0.8527478010727135, -0.1190366594999854: 0.4007671175484125, -0.2042810895467515: 0.2648143715540994, -0.2020047152493137: 0.296602476637973, -0.1096302179115897: 0.7815076095377649, -0.0402586757625513: 1.178959839997653, 2.2618193599287477: 2.6609784572167405, -0.1873356049461633: 1.726524889667015, -0.1905611115777871: 0.9386475995934032, 0.9567469673769764: 1.6622556864190603, -0.1189286281773952: 1.1714642764987553, -0.199404246984105: 1.3864783525103226, 0.2377985155387222: 1.901999382961011, -0.1955305524169396: 0.3469625491620661, 0.3364156800175554: 4.6752006545906974, -0.2117506838515646: 0.2050366539023548, 0.1933050436776148: 0.5351952623958193, -0.0402123766242983: 2.2720290085107364, -0.1938020512554952: 0.343359266158716, -0.0534847962568173: 1.0878983158461015, -0.0755000364961062: 2.5072176507657944, -0.2122985569875581: 0.2905961389106034, 1.934553901187605: 3.691732686248401, 0.1316886071859495: 2.1974541288858327, 0.4849433155330939: 5.40512781092603, 1.7394724821587018: 1.8946954141381025, -0.2071979352566888: 0.2203774753904387, -0.1093987222203248: 1.4519547973746028, 2.821259563962466: 12.384353706874832, 0.1998949543556213: 3.7776026357354535, 0.1059231367481698: 1.8938715905504049, -0.1864404882732725: 0.4876291680466989, -0.0388928511840886: 2.3338618758065364, -0.1057256572522556: 1.318481888299282, 0.1886442637601488: 5.257090751319649, -0.1993039321845569: 1.133063284245933, -0.1985631459725093: 0.3450976796236576, 2.3374258526958527: 14.799762667739191, -0.0207590220350073: 2.346189031642958, 0.0255246831718816: 4.413664895625509, 0.4048303733093657: 2.563008287934855, 0.1157848531960531: 0.6719129485674799, -0.1550882551529673: 0.5998803196501429, 3.708590265103743: 9.974087160024537, -0.1789708939684594: 1.0567327122123134, -0.1411367814927381: 2.1332170254222205, 3.665879310065375: 6.233167305829428, -0.1756064899220767: 0.2263982494921767, -0.201356527313772: 1.03090073462532, -0.1746805071570172: 0.7582244767155273, -0.1181492593501368: 2.635395726197597, -0.1605206873746495: 1.587881625926134, -0.198740626002479: 0.1899500510343452, -0.1960938585990174: 0.1211767889026885, 0.5815927666361758: 2.721680380488156, -0.1984859807420877: 0.4973336091451641, -0.0866658453381149: 2.7222042179343573, -0.2027763675535299: 0.0716978109580215, -0.215385166204423: 2.5368040225613684, 0.0234412219504978: 1.7712224917815935, -0.0049170002294483: 2.52922582225544, -0.1323090791325045: 1.1265697002920136, -0.15074385268023: 0.80537506815761, -0.1985245633572985: 0.2207294000238908, -0.1515077884614041: 0.8587553622641394, 1.5308717147599302: 2.345175683150218, -0.1749120028482821: 0.955369301948036, 0.0001218893170835: 0.9635140474400706, -0.1142215491216762: 1.363569778029646, -0.1725738963665069: 0.6822305780869432, -0.1717327953549113: 0.5454053640840243, 1.7730316408690652: 17.743261192126102, 1.730590764137173: 4.646258535485707, 0.8209438783579636: 5.719723916523254, -0.1986017285877201: 1.101353846801961, 1.4995349146857095: 5.362271326494548, -0.201449125590278: 0.3456856202277816, -0.1694487045344312: 0.9092629858025488, 1.7181825950853762: 4.962219902990795, -0.1932001624582066: 0.5374646077340063, -0.1365068676674407: 0.3252994712414023, 1.031064800796041: 1.6842972620458858, -0.1342922255543402: 0.9061465439451436, -0.1255031058093174: 1.770420798091095, -0.2024136909705483: 0.2023715338215369, 0.0169516260720393: 1.9643597923983307, -0.1580514000011576: 0.9406168010358882, -0.2119821795428294: 0.4536266577130432, -0.1714550005253934: 0.2785502343470596, -0.1809849064824638: 0.407810799385389, -0.2005848750095558: 0.1209910153141137, -0.2016034560511212: 0.2063003569552951, 0.0600947064007683: 1.9229127424843817, -0.1497792872999597: 0.4450777226984249, -0.1984473981268769: 0.2787286328882862, 8.352617611045192: 4.8654536003454805, -0.1238980690165476: 1.4926703034626714, -0.166678472762295: 0.1262374808966676, -0.1873510379922476: 0.1261587344485362, -0.1512531432010127: 0.5871354784083933, -0.2036020355190412: 0.5243887147708136, -0.2049601435744618: 0.8861180622243756, -0.1244150760603725: 0.8900558379606479, -0.1989181060324488: 0.2885165294255853, -0.2061407715999126: 0.1902334379774574, -0.1740246026984334: 0.1625790593225245, -0.2095746243536748: 0.1423666047891372, -0.1671414641448247: 0.6001027074003178, -0.0687094962190034: 0.1782852371411144, -0.1907694576999255: 0.2776587919629298, -0.0523504673696195: 0.55216245316165, -0.1544014846022149: 0.5739831229613435, -0.1062272312499961: 0.5345141124238157, -0.1886165477711622: 0.4256787875780987, -0.2019044004497656: 0.2679804360129754, -0.0214920917240128: 0.8957555010935551, -0.2051762062196424: 0.2676926079212444, -0.2002762140878693: 0.1504980001578924, -0.1438221315114105: 0.3387111118152333, -0.2013796768828985: 0.2076211634471206, -0.1871195423009827: 0.3427953967505443, 0.2389482774720044: 0.3356394908239053, 0.9624726274742608: 1.0519201957607711, -0.1289678246552482: 0.3307328488038834, -0.1722343693526518: 0.1927350290616813, -0.1920889831401352: 0.3083179710346354, -0.1889252086928487: 0.1782069866629198, -0.2125223361557808: 0.2218938881453536, -0.2050527418509677: 0.3903447218983285, -0.2099295844136143: 0.1662367328729951, -0.1776436520052075: 0.1452414182885844, 0.5733206539349778: 0.358808025433581, -0.1743255470970778: 0.8302176465714609, -0.060792343577745: 0.2936341117821906, -0.2047749470214499: 0.08246094166982, 0.0357645092488308: 1.281302552504723, -0.2021281796179883: 0.5506182630350278, -0.2022902266018737: 0.3422325163675179, -0.1962096064446498: 0.2519744975332488, -0.1052780989158102: 0.2159482193175722, -0.2028380997378672: 0.136969212586605, -0.2043273886850045: 0.1760928387224052, -0.2006620402399774: 0.0349889943072974, 0.0168667443185756: 0.7413282549830363, -0.2005694419634715: 0.3654658035549287, -0.2065960464594002: 0.2681740672947502, -0.0985029916847917: 1.0828170651297253, -0.2081625006369591: 0.1559086352154764, -0.1851286793561049: 0.3660880512471456, -0.1046839266415637: 0.2733275936564651, -0.1528273139016138: 0.2965424816306184, -0.1940644130389287: 0.261338973863172, -0.2115269046833418: 0.1377673729293092, -0.1641165871122971: 0.2573963941824346, -0.194172444361519: 0.1527612778162017, -0.198022989359558: 0.3948854217092149, -0.1195845326359789: 1.0832521125456231, -0.1794107357818627: 0.1297564767813031, -0.2093431286624099: 0.1131675109440255, -0.2113880072685829: 0.0802304165000062, -0.1758611351824681: 0.4618592652059295, -0.2060713228925332: 0.3917843267677678, -0.1859543473216162: 0.2991321985051072, -0.1505432230811338: 0.3861783155439724, -0.1734998791315664: 0.0745452070225574, -0.1905842611469136: 0.1925795591489238, -0.2028843988761202: 0.2116698132788301, -0.2096440730610543: 0.218969427415542, -0.1252716101180525: 0.7908224515725772, -0.208679507680784: 0.307416178333213, -0.1177711497210708: 0.5083685261260712, -0.2116040699137635: 0.1153176632806114, -0.2044045539154261: 0.1633901377168096, -0.1886628469094152: 0.4697904295192464, 0.3735630219425244: 0.6153841704355245, -0.2066654951667796: 0.2288557919291477, -0.1965414169354628: 0.2248546100933551, -0.2108632837017159: 0.1574885878912397, -0.177427589360027: 0.1741620647512371, -0.1986866103411839: 0.3917010264910153, -0.2059787246160272: 0.1294632727704282, -0.1990647199702498: 0.3148323604525689, -0.2004073949795861: 0.1238494692548651, -0.1888094608472163: 0.2176997861289084, -0.2004614106408812: 0.82612320403412, -0.1855839542155924: 1.0048078253526385, -0.1816330944180054: 0.4396609007641839, -0.2016111725741634: 0.3496838180326446, 0.0541066845200505: 2.4852160817945417, -0.1357274988401823: 1.0915257983803806, 2.267452421749525: 2.6224213659644744, 0.2613184777712327: 2.4940074518844724, 0.1280695578791754: 1.2601886265258873, -0.1965799995506736: 0.4855065326307717, 0.6239641946606885: 2.832253922184766, -0.1702049237925632: 1.0311323137012676, 0.3702063344191838: 1.254447505059618, -0.1804910490077654: 0.6489125540730577, -0.2111410785312337: 0.1229818080693264, -0.2124297378792748: 0.6292128541451049, -0.1272084574016352: 1.3553361054030415, -0.2100067496440359: 0.1564128894171001, -0.064789502513585: 1.6370367538175985, -0.022742168456843: 1.3783124475405764, -0.109360139605114: 1.60608347786179, -0.0259831081345512: 2.3138686303797806, -0.0724442933714099: 1.8665629543254387, 0.259528244425451: 0.5297802098548449, -0.2097675374297289: 0.2475457653473821, -0.1309818371692526: 0.6563853664786747, -0.166632173624042: 0.7684262633395329, -0.2000292853505201: 0.2509080924849584, 2.0615292878463847: 7.73371728391588, 0.3668573634188854: 3.4430925495651814, 5.322701688971067: 8.76193931178312, 0.2353755273034833: 3.471620002117429, 0.126626568070291: 4.522242280198373, -0.1398867047599078: 0.7699965549491035, -0.2110021811164748: 0.1052303246700977, 0.0904515080486345: 2.7311797579278414, -0.1310358528305477: 0.7279169534677422, -0.2001758992883212: 0.3626479440892552, -0.2070436047958456: 0.0961443417840423, -0.1869034796558022: 1.2086352411063646, -0.1939023660550433: 0.97437505925963, -0.1959240950920898: 0.9318296926902114, -0.2081702171600013: 0.4284606727642385, 0.2951708643571983: 2.802664302185517, 0.1303690817457397: 1.1843963680798144, -0.1583986435380549: 2.4236342866464264, -0.2103771427500597: 0.4819590796220351, -0.027117437021749: 1.481944410395557, -0.2128464301235516: 0.1676324966046987, -0.1732143677790064: 0.9524808976522252, -0.2142971364554781: 0.2933070807635623, -0.1853678915704119: 0.1245116219159647, 3.0656186991386174: 6.64505751082315, 0.1955196857907154: 4.387576682155847, -0.2155163470961397: 0.6566505752949726, -0.2028689658300358: 0.2918056932017739, -0.1477729913089976: 0.6517145156209719, -0.1415689067830991: 0.8861972956531676, 0.1275602673583927: 3.1555066278718567, -0.1364374189600612: 0.5982029859668789, -0.1872893058079103: 0.7309368095425927, -0.0853926190361581: 2.3535056530169176, -0.1204410666936589: 0.7508631764128227, 3.4335425177889123: 7.189658183423214, -0.1376720626468072: 0.1900767153594008, -0.1940103973776336: 0.1995099727043117, -0.1977066119148293: 0.361183642704334, -0.1834078947177027: 0.6638711575208649, -0.0641644641471699: 1.337246397816818, -0.1777671163738821: 0.2218084447220303, -0.1308506562775358: 2.3415370455214504, -0.0985878734382555: 2.444455381823192, -0.2123294230797267: 0.2050948221509496, -0.1999135375048877: 0.7477299128479981, -0.2150610722366522: 0.4646951798346902, -0.0711787835924953: 1.6353815827445883, 1.5177536255882544: 8.031303048531491, 0.005947864213916: 2.0210692925890816, -0.120680278907966: 1.5889553914041077, -0.1975368484079017: 0.2328280197369484, 1.3170700108307416: 12.343830996312256, -0.0039215687570094: 1.9055673774413489, -0.2135409171973462: 0.4063558019539468, 0.8943897446732636: 10.999804114288027, -0.0320714448148172: 0.8958163408529556, 0.189747726555178: 2.001016615934676, 0.0153774553714383: 1.2950061473297565, -0.1789631774454173: 1.3233016180631465, 19.426839041066504: 9.061709822282737, 7.683417583259756: 20.67094906683988, 0.0219364999572761: 3.603419587222776, -0.210631788010451: 1.171907152524515, -0.0453670140164627: 2.8167057512588416, -0.1506512544037241: 1.2332503001068524, -0.1019368444385539: 0.930726068103358, -0.1286282976413931: 2.1276295828660494, -0.0203809124059414: 4.779802915427702, -0.1282116053971163: 1.0633823874852613, 8.617749626250843: 7.95527962003704, 0.0101610857949366: 2.472736900525669, -0.2012870786063926: 0.0637564135629939, 1.057416726985025: 0.9796398837144976, -0.0966741757237993: 1.273158865961782, -0.058176442266452: 0.8631104137072172, 0.1242653120193894: 2.4485352871807207, -0.1937171695020314: 0.1402275998961898, -0.0223949249199457: 2.275834539953029, -0.0971988992906663: 2.462106767121077, -0.1493703115787251: 0.7166362583445164, -0.203208492843891: 0.1966408789879943, -0.1849126167109243: 0.533023615484219, -0.0880856855778727: 3.4272775235256834, 0.3408449642437565: 2.5197418860770733, -0.2154931975270132: 1.0936595446384072, -0.2078769892843991: 0.2490136548448389, -0.0357985254441815: 1.073031202746122, -0.1800820732865308: 0.2005191760825201, 2.0432719943286286: 4.036091508593255, 0.1125979291796401: 1.840116100923992, -0.1871118257779406: 0.2697475232658112, -0.1994351130762736: 1.5869254383810143, -0.203355106781692: 0.3560437682531261, -0.2117198177593959: 0.2502188129848705, -0.2085251772199407: 0.1405490361450838, -0.1254182240558536: 5.5401764498640205, 0.2984889692653281: 3.689364405964619, 0.0556577056515251: 3.875382594549434, -0.2095591913075905: 1.9728492381428828, -0.1797734123648443: 0.4583746445647531, -0.0285527103075912: 2.3533939804666364, -0.0309911315889145: 2.007867727235256, 0.8689175021110861: 3.030950877756294, -0.0802611312131203: 1.996515345212336, -0.1799663254408983: 0.8714492030806625, -0.1886011147250779: 0.5080094381823885, -0.1299246735124763: 0.0976451243895638, 0.0973809457404962: 1.100334278532179, 0.1221895673210478: 1.825940430067816, 0.3904236247896488: 2.842692134365125, -0.2054694340952445: 0.4374681412116199, -0.1026776306506015: 1.2204108967000808, -0.210392575796144: 0.1667830466757928, 4.774195798088092: 4.636226689917831, -0.082244277634956: 5.265917889201437, 0.1007916489251319: 3.2919737802887035, 0.5802886742420503: 2.5212466497672312, -0.1629436756098885: 0.5842834088602004, -0.1734690130393978: 0.5902616512153042, -0.1764475909336724: 1.9655823332311408, -0.1657987891354885: 0.6410023247635569, -0.15131487538535: 1.2455103260736031, 0.4853985903925815: 8.704714229568554, 2.524474371237865: 3.0122615369875505, 0.6277298579052637: 2.4895690416541143, -0.1917263065571536: 0.5051906774541723, -0.1972590535783839: 0.3364756111143016, -0.2072210848258153: 0.1943932700720727, 0.0900965479886951: 2.661135411536607, 0.3536235264015772: 1.2712767083280665, -0.2118124160359019: 0.6303495936030852, 0.2727775144888436: 2.3408501532627137, -0.1684918556772031: 1.4199471123483134, -0.1679748486333783: 0.4992206816108991, -0.0839264796581473: 1.4605069862243878, -0.2052765210191904: 0.7495488186736179, 3.1697531775925967: 3.613770285513272, -0.1238131872630839: 1.0584755508863264, -0.1017284983164156: 1.49822722978868, -0.1579125025863987: 0.4374399155977479, 0.6853800015532576: 3.389003430582388, 6.053294374079944: 1.4442330793912357, -0.2132091067065332: 2.5201694273524184, -0.1294771151760309: 0.4582010960562628, -0.0904546581518166: 0.5209170494877908, 0.9128939669283688: 4.59732471922253, 2.676219796861985: 2.6987770367033894, 1.6668754333780396: 1.9958236451448264, -0.1866951335336638: 0.573901576135451, -0.1929378006747731: 0.5608557327993239, -0.1476263773711965: 2.346724938542795, -0.1907231585616725: 0.8150666621850409, -0.199643459198412: 0.9650793077607112, 13.004091401722327: 5.463514762118479, 1.370082524130396: 4.552027806513216, -0.1877445806673979: 1.053891105234984, 0.2881333953427464: 3.119761342265196, 0.33454056491831: 1.5191572210472737, -0.2082319493443386: 0.3257885248983944, -0.204065026901571: 0.7377882574066723, -0.1876905650061028: 0.8290015330280316, -0.1776745180973762: 0.2519471448902191, -0.2071284865493094: 0.6482682440872822, -0.0344018347735502: 1.3240452667872349, -0.1858771820911946: 0.5043593791590061, -0.1040125891368956: 1.6661370968228129, -0.0307056202363544: 1.303073138496042, -0.2103385601348489: 0.1864053681407341, -0.2070050221806348: 0.2474009682446571, -0.1639313905592853: 0.567871849748198, -0.2097289548145181: 0.2348017991497184, -0.165204616861242: 1.3645446279550526, -0.187559384114386: 0.1742797879923188, -0.2060558898464488: 0.4806618170619142, -0.2082165162982543: 0.1834239628948914, 0.5680348356510967: 3.2938151124245567, -0.1889020591237222: 0.5217520073790678, 0.0786220782249998: 0.9478964280777402, -0.160921946572842: 0.7477318373661942, -0.1920658335710087: 0.3943246860345502, -0.1778597146503881: 0.6061230618392472, -0.1037579438765042: 2.441073538988105, -0.208872420756838: 0.1014037367282875, -0.2111487950542759: 0.163204251808707, 1.308666717237827: 1.4148822660600668, -0.1976140136383234: 0.2868386881615308, -0.181424748295867: 0.4648989067530751, 0.1325682908127559: 1.7586173119130122, -0.064264778946718: 7.432058376687005, 0.0777655441673198: 1.0892503358990815, 0.3297717536782538: 1.7617364741512305, -0.169248074935335: 0.890143133867169, -0.1772192432378886: 0.6523875916886891, -0.0746743685305948: 3.0260315093500174, 0.107597622248319: 1.367002765544778, -0.1181878419653476: 2.0503088171943817, -0.2017577865119645: 0.7479602937497578, 0.0270139721190189: 0.4791509330033108, -0.0825683716027268: 0.5425597856365469, 0.792709120546692: 2.7881644496148104, 0.968522381539316: 5.671334924987785, -0.0536699928098292: 3.786681346750481, 0.9071065746467468: 2.2197680286573096, -0.1480893687537262: 1.649567371295366, -0.1991804678158823: 0.3322729489669224, -0.1882770207573071: 0.1597862175641767, 0.3078336786693865: 1.0156194812579429, -0.0570421133792541: 0.6971309960578264, 0.6011155699328461: 1.4855437977076051, -0.2098447026601505: 0.0743661502860251, -0.1184733533179076: 0.6813618454218432, 1.0362348712342897: 1.5066276823781048, -0.2115037551142154: 0.0674304990868935, -0.1804138837773438: 0.6274201196101731, 0.0844557696448745: 1.2339238290066925, -0.1694641375805156: 0.9291109726873734, -0.2138727276881592: 4.304526019465992, -0.2107089532408727: 0.3336814024986965, -0.0434918989172173: 1.4078266283384533, -0.2144514669163213: 0.2009156670306667, 2.7404521346649435: 5.253489724313521, -0.1898280418887817: 0.7603319289622088, 3.0865999252902565: 9.425690268838608, -0.0796360928467051: 1.2317064651204697, -0.0697898094449061: 2.030921205074352, 0.2758564071826663: 1.6052308531428023, -0.177095778869214: 1.4166612440588724, 0.2211231092446096: 2.5255257288648973, 0.0190813864316761: 0.8349128287500366, -0.2030464458600056: 0.4087963007952135, -0.1801129393786994: 0.3622816520780698, 0.0299848334902513: 1.1818362284668755, -0.1454117352580959: 0.8278673717441627, -0.1700120107165091: 1.8109153830314573, 5.1550910919722615: 8.608486636332604, 0.580080328119912: 1.3047048895904434, -0.1126319453749908: 1.9665156387624203, -0.1705367342833761: 0.3447208064717575, -0.1504892074198387: 0.7024038061878255, -0.2046823487449439: 0.1229032469721386, 0.9839940102388514: 2.197335879019556, -0.1967266134884747: 0.2959760243162282, -0.1459364588249629: 0.6109062610659076, -0.2076069109779234: 0.1145733009579808, -0.1972127544401309: 5.121104267213483, -0.2012639290372661: 0.101765649455176, -0.1699271289630453: 0.4967437818221845, -0.2048675452979559: 0.4301393000488111, 0.8835325967529414: 3.0582695068889887, 0.4541852546870353: 4.818744967324767, -0.1206879954310082: 3.0676373756815893, -0.1042595178742448: 1.6172068813313434, -0.1927294545526347: 0.9381018004926568, -0.142656936532044: 0.7728224319145042, 0.6418896776876314: 8.261640140663678, -0.1165287895112827: 0.4649560745722343, 0.5164112964990314: 2.3067702475693173, 0.3859248918560682: 5.889538007382979, 0.0350931717441627: 1.8949633036654625, -0.0238764973440409: 1.1963803089502765, -0.2119513134506608: 2.6463725054957368, -0.0519414916483849: 1.1144979498926804, 0.3878848887087774: 1.1853665432210334, -0.2059247089547321: 0.1861217867569543, -0.1800743567634886: 1.6408096893347588, -0.1795727827657481: 1.6648754526246994, 0.2727543649197171: 2.319038681670268, -0.0279508215103025: 0.6084873857687866, -0.1259815302379314: 0.8394291577535655, -0.1053012484849367: 0.2177354258211572, 0.8627982993386515: 2.006507204436816, -0.1625346998886539: 1.589023215478988, -0.205114474035305: 0.2279515526454645, -0.1876365493448076: 0.1622729152803979, 0.5655886978467313: 2.260676293857516, -0.2145440651928273: 0.2674628085140068, -0.2122136752340943: 1.3139170958948283, -0.0615022636976239: 1.6875060131061244, 0.0704348472772657: 2.601948634972986, -0.1166136712647465: 1.0068915918240886, -0.1384205653818969: 0.7701872694393789, -0.1809386073442108: 1.2042586358665583, -0.2077226588235559: 0.6487180339648532, -0.0517254290032044: 1.2698051706001787, 1.7513096285053786: 3.0421905293507785, -0.2126149344322867: 5.039317033983736, -0.1693638227809674: 0.8876392379520639, -0.2121750926188835: 0.3133921988654338, -0.0742499597632759: 1.376346461961503, -0.1787625478463211: 0.8535173795707298, -0.1077705358584286: 1.346212808527515, -0.168962563582775: 1.5405313736015949, -0.1547024290008592: 1.203674596173541, -0.127910660998472: 1.8001979793639458, -0.2029924301987104: 0.1164537847481986, -0.160019113376909: 2.9883169764628184, -0.1706910647442193: 0.3262616845455764, -0.1349172639207553: 0.7457291581951118, -0.1705213012372918: 0.3703929161348261, -0.1428652826541824: 2.7005705254889256, -0.2066423455976532: 0.1620882114484436, -0.1020448757611442: 3.003116358791777, 0.3411150425502322: 2.8692113308230867, -0.1128557245432135: 0.8441957936742389, -0.2137261137503581: 0.2457444970097822, -0.1323399452246731: 1.1125237506793504, 3.558318695380675: 4.9891460250054935, -0.1901444193335103: 1.1227853293372838, -0.1882230050960119: 0.5693341759741949, -0.2020124317723558: 0.2013226688551662, -0.1842875783445092: 0.4186173988842851, 0.0317442007438643: 0.5258509908339757, -0.1969426761336553: 0.2681077443843093, 0.0593076210504678: 0.4950086908643614, -0.1014815695790664: 0.4066988389461217, -0.208965019033344: 0.3213714363975711, -0.0672510733640347: 0.4581537595019127, -0.2077149423005137: 0.2067337177611576, -0.1948977975274823: 0.2717995175286297, -0.1824047467222216: 0.19483083419705, -0.2024985727240121: 0.3186529692608299, -0.2130856423378586: 0.5502730103547542, -0.1814864804802043: 0.3954989268921896, -0.1914948108658887: 0.2560399596734911, -0.164108870589255: 0.3222774802603094, -0.1622106059208831: 0.4988896321458006, -0.1498024368690862: 0.4188537739127739, -0.2136952476581894: 0.6581115177594188, -0.1973825179470585: 0.3079237027456425, -0.1878217458978195: 0.3552796843157583, -0.2057086463095515: 0.169465180189738, -0.1262053094061541: 0.5301793988490315, -0.1759691665050583: 0.5128835247761754, -0.2050450253279256: 0.0755494956198693, -0.2093585617084943: 0.6803233813494812, -0.1887785947550476: 0.2125887648686054, -0.2062642359685872: 0.1250912955854912, -0.1903373324095644: 0.3583535597445374, -0.1871889910083622: 0.0799368390254118, -0.2093739947545786: 0.0791926322834356, -0.2140579242411711: 0.4691716102164484, -0.206302818583798: 0.1088285737049206, -0.2004768436869655: 0.1607994252632832, -0.071163350546411: 0.8606438030809362, 0.1305311287296251: 0.3562485236433943, -0.201495424728531: 0.218040771605088, -0.2101379305357527: 0.0958993997304227, -0.2154005992505073: 0.3133316838001506, -0.1896814279509806: 0.7578197072304022, -0.21191273083545: 0.0561034321332215, -0.1992113339080509: 0.1492504934613511, -0.1916414248036898: 0.178343530382541, -0.2039029799176856: 0.1468205427553324, -0.2009552681155796: 0.2813722229908884, -0.1037425108304199: 0.4827981267384631, -0.1827905728743297: 0.5265431729019221, -0.2017114873737115: 0.1733048068824189, -0.2079541545148207: 0.1448402215691397, -0.2044585695767213: 0.1726096267983712, -0.149223697640924: 0.3692466933310361, -0.2085560433121094: 0.1087579966834945, -0.2081933667291278: 0.4929088385326851, -0.1118371435016481: 0.3954366589406149, -0.1697033497948226: 0.4541015934025081, -0.1793258540283989: 0.1486477403082775, -0.2007006228551882: 0.7814172053521039, -0.2114960385911732: 0.4380229771648919, -0.1381504870754213: 0.1663431806035791, -0.18261309284436: 0.7194601710021467, -0.2098832852753613: 0.0431314529762078, -0.1692557914583772: 0.3239279519684983, -0.1905996941929979: 0.1194559404139, -0.2104543079804813: 0.0827275211788311, -0.1324634095933477: 0.6663211086486996, -0.1124004496837259: 0.3536583103763916, -0.2028458162609094: 0.0796607109293795, -0.2026837692770239: 0.3280115702483065, -0.2049061279131667: 0.5946418492303819, -0.1919037865871233: 0.4089502425510032, -0.1493857446248095: 0.2051305251050564, -0.1966494482580531: 0.2762469297783906, -0.1899206401652876: 0.1553960109003101, -0.2035094372425353: 0.2921034735596853, -0.160829348296336: 0.4831856911888746, -0.0998996823554231: 0.6638310300764154, -0.2043351052080466: 0.2750440164735817}
ttt_dict = {1.0834998504483295: 7.225253856577795, -0.8533779196331812: 0.6638310300764154, -0.0893606367863633: 0.5991052652631076, -0.1190549014518728: 1.5405313736015949, -0.2722793021216385: 0.3186529692608299, 0.4845260711377215: 0.7814172053521039, 1.7741461775959142: 0.4380229771648919, 0.8940662771610375: 0.3133921988654338, -0.1394612140916263: 0.2750440164735817, -0.1238291756237912: 0.0561034321332215, 3.4755159478555995: 1.1963803089502765, 1.3075777715723294: 4.9891460250054935, 0.7007274997195408: 0.4541015934025081, 1.9533840757973415: 1.1125237506793504, 0.926327851964959: 0.5769111392862717, -0.0568035532389721: 0.0248518492603745, 0.1246375683789556: 0.4186173988842851, -0.1412342665524432: 1.8949633036654625, 1.439131531054333: 2.9883169764628184, -0.3981572987332914: 2.8692113308230867, 0.5261017822864409: 0.5128835247761754, 0.0725120704011422: 1.203674596173541, 0.8194222648074482: 0.3988094303537475, -0.0430879575505151: 0.073582885031982, 0.2317027551953867: 0.8876392379520639, 0.102405834640302: 0.3133316838001506, 2.32712782394612: 0.1622729152803979, 1.9595012314743936: 1.0068915918240886, 0.4643242455609596: 1.376346461961503, 0.5333486042992874: 0.3336814024986965, 0.3202992689811427: 0.4831856911888746}

In [ ]:
def twitter_fillna(data,player_twitter,team_twitter,playerId,teamId):
  data = data.copy()
  # 先把所有的 player_twitter 项给 fillna 一下
  # 对于那些有记录的 palyer 来说，由于记录发生在每月的1号，所以这一个月的 player_twitter 全部按照1号的填充
  # 可能有些人有的月份没有 player_twitter 记录，那么就按上一个月的记录延续，如果是开头几个月没有，那么就按最早的记录填充
  # 所以先 forward fill，在 back fill
  data[player_twitter] = data.groupby(playerId)[player_twitter].fillna(method='ffill')
  data[player_twitter] = data.groupby(playerId)[player_twitter].fillna(method='bfill')
  # 但是这么操作一圈以后，有些人他们一条 player_twitter 记录都没有，只能全部-1了
  data[player_twitter] = data[player_twitter].fillna(-1)

  # 接下来处理 team_twitter
  # 对于那些有 teamId 的行，team_twitter 的缺失主要是由于 team_twitter 的记录都在每个月的1号
  data[team_twitter] = data.groupby([teamId],)[team_twitter].fillna(method='ffill')
  # 然后把没有 teamId 记录的 player_date，全部 team_twitter =-1
  data[team_twitter] = data[team_twitter].loc[data[teamId].isnull()==True] = -1
  return data

In [ ]:
def var_type_judge(data):
  data = data.copy()
  result = pd.DataFrame(index=data.columns,
                        columns=['variable type'])
  for c,n in data.iteritems():
    if data[c].unique().shape[0] == data[c].shape[0]:
      result.loc[c]='ID'
    elif data[c].dtype == 'int64' or data[c].dtype == 'float64':
      result.loc[c]='numerical'
    elif data[c].dtype == 'O':
      result.loc[c]='catagorical'
    else:
      result.loc[c]='?????????'
  return result

In [ ]:
def trim_fraction(text):
    text = str(text)
    if '.0' in text:
        return text[:text.rfind('.0')]
    return text

In [ ]:
X_columns = ['playerId', 'teamId',"player_twitter","team_twitter",
           'status', 'awardId', 'seriesDescription',
           'Current_winning_percentage', 'Score', 'home', 'jerseyNum',
           'positionCode', 'battingOrder', 'gamesPlayedBatting', 'flyOuts',
           'groundOuts', 'runsScored', 'doubles', 'triples', 'homeRuns',
           'strikeOuts', 'baseOnBalls', 'intentionalWalks', 'hits', 'hitByPitch',
           'atBats', 'caughtStealing', 'stolenBases', 'groundIntoDoublePlay',
           'groundIntoTriplePlay', 'plateAppearances', 'totalBases', 'rbi',
           'leftOnBase', 'sacBunts', 'sacFlies', 'catchersInterference',
           'pickoffs', 'gamesPlayedPitching', 'gamesStartedPitching',
           'completeGamesPitching', 'shutoutsPitching', 'winsPitching',
           'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
           'groundOutsPitching', 'runsPitching', 'doublesPitching',
           'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
           'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
           'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
           'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
           'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
           'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
           'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
           'inheritedRunnersScored', 'catchersInterferencePitching',
           'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
           'assists', 'putOuts', 'errors', 'chances', 'divisionRank', 'leagueRank',
           'totalWins_onTHEseason', 'totalLosses_onTHEseason',
           'homeWins_onTHEseason', 'homeLosses_onTHEseason',
           'awayWins_onTHEseason', 'awayLosses_onTHEseason',
           'positionName', 'positionType']

df_X = ['playerId', 'teamId', 'status', 'seriesDescription',
       'Current_winning_percentage', 'Score', 'home', 'jerseyNum',
       'positionCode', 'positionName', 'positionType', 'battingOrder',
       'gamesPlayedBatting', 'flyOuts', 'groundOuts', 'runsScored', 'doubles',
       'triples', 'homeRuns', 'strikeOuts', 'baseOnBalls', 'intentionalWalks',
       'hits', 'hitByPitch', 'atBats', 'caughtStealing', 'stolenBases',
       'groundIntoDoublePlay', 'groundIntoTriplePlay', 'plateAppearances',
       'totalBases', 'rbi', 'leftOnBase', 'sacBunts', 'sacFlies',
       'catchersInterference', 'pickoffs', 'gamesPlayedPitching',
       'gamesStartedPitching', 'completeGamesPitching', 'shutoutsPitching',
       'winsPitching', 'lossesPitching', 'flyOutsPitching', 'airOutsPitching',
       'groundOutsPitching', 'runsPitching', 'doublesPitching',
       'triplesPitching', 'homeRunsPitching', 'strikeOutsPitching',
       'baseOnBallsPitching', 'intentionalWalksPitching', 'hitsPitching',
       'hitByPitchPitching', 'atBatsPitching', 'caughtStealingPitching',
       'stolenBasesPitching', 'inningsPitched', 'saveOpportunities',
       'earnedRuns', 'battersFaced', 'outsPitching', 'pitchesThrown', 'balls',
       'strikes', 'hitBatsmen', 'balks', 'wildPitches', 'pickoffsPitching',
       'rbiPitching', 'gamesFinishedPitching', 'inheritedRunners',
       'inheritedRunnersScored', 'catchersInterferencePitching',
       'sacBuntsPitching', 'sacFliesPitching', 'saves', 'holds', 'blownSaves',
       'assists', 'putOuts', 'errors', 'chances', 'divisionRank', 'leagueRank',
       'totalWins_onTHEseason', 'totalLosses_onTHEseason',
       'homeWins_onTHEseason', 'homeLosses_onTHEseason',
       'awayWins_onTHEseason', 'awayLosses_onTHEseason', 'awardId',"player_twitter","team_twitter"]
set(X_columns)==set(df_X)

In [ ]:
def preprocess(data, test_y):
    #preprocess submission y
    merge_start = test_y.copy()
    merge_start.drop(columns=["date_playerId"], inplace=True)
    merge_start["engagementMetricsDate"] = merge_start['date'].copy()
    merge_start["date"] = pd.to_datetime(merge_start['date'], format="%Y%m%d")
    merge_start["date"] = pd.DatetimeIndex(merge_start["date"]) - pd.DateOffset(1)
    merge_start['date'] = merge_start['date'].astype(str)
    merge_start['engagementMetricsDate'] = pd.to_datetime(merge_start['engagementMetricsDate'],format='%Y%m%d')
    merge_start['engagementMetricsDate'] = merge_start['engagementMetricsDate'].astype(str)
    
    rosters = merge_nested_form(data, 'rosters')

    # rosters 原本一共有6列：
    # playerId	gameDate	teamId	statusCode	status
    # 现在仅保留有用的4列
    rosters = rosters[['playerId','gameDate','teamId','status']]

    # 并且 pd.isnull(train['rosters']).sum() 的结果为0，也就表示，train['rosters'] 这一列每一行都有 value
    # 所以，gameDate 应该就是 date
    rosters.rename(columns={'gameDate':'date'},inplace=True)
    
    #     playerTwitterFollowers = merge_nested_form(data, 'playerTwitterFollowers')
#     print(playerTwitterFollowers)
#     # playerTwitterFollowers 原本一共有6列：
#     # date	playerId	playerName	accountName	twitterHandle	numberOfFollowers
#     # 现在仅保留有用的3列
#     playerTwitterFollowers = playerTwitterFollowers[['date','playerId','numberOfFollowers']]

#     # 嵌套表格以'date'为列名称的，数据格式好像全部都是 datetime，需要全部转换成 str
#     # 比如 transactions，playerTwitterFollowers，teamTwitterFollowers 表
#     # 但是以'gameDate'为列名称的，数据格式好像全部都是 str
#     playerTwitterFollowers['date'] = playerTwitterFollowers['date'].astype(str)    

#     playerTwitterFollowers.rename(columns={'numberOfFollowers':'player_twitter'},inplace=True)
    
#     teamTwitterFollowers = merge_nested_form(data, 'teamTwitterFollowers')

#     # teamTwitterFollowers 原本一共有6列：
#     # date	teamId	teamName	accountName	twitterHandle	numberOfFollowers
#     # 现在仅保留有用的3列
#     teamTwitterFollowers = teamTwitterFollowers[['date','teamId','numberOfFollowers']]

#     # 嵌套表格以'date'为列名称的，数据格式好像全部都是 datetime，需要全部转换成 str
#     # 比如 transactions，playerTwitterFollowers，teamTwitterFollowers 表
#     # 但是以'gameDate'为列名称的，数据格式好像全部都是 str
#     teamTwitterFollowers['date'] = teamTwitterFollowers['date'].astype(str) 

#     teamTwitterFollowers.rename(columns={'numberOfFollowers':'team_twitter'},inplace=True)

    games = merge_nested_form(data, 'games')

    # gameType
    '''
    [{"id":"S","description":"Spring Training"},
     {"id":"R","description":"Regular Season"},
     {"id":"F","description":"Wild Card Game"},
     {"id":"D","description":"Division Series"},
     {"id":"L","description":"League Championship Series"},
     {"id":"W","description":"World Series"},
     {"id":"C","description":"Championship"},
     {"id":"N","description":"Nineteenth Century Series"},
     {"id":"P","description":"Playoffs"},
     {"id":"A","description":"All-Star Game"},
     {"id":"I","description":"Intrasquad"},
     {"id":"E","description":"Exhibition"}]
    '''

    # 只有10场比赛存在resume，所以相关变量可以删除了
    # 只有116场比赛 istie 那一列是1，可以删除

    # 改一下日期的 column name，方便后面 merge
    games.rename(columns={'gameDate':'date'},inplace=True)
    
    # 把 game 表格分程两部分，主场球队的信息，和客场球队的信息
    # 两部分都包括比赛的编号和类型
    # 仅摘取我们认为比较有用的信息
    games_home = games[['seriesDescription','date','homeId','homeWinPct','homeScore']]
    games_away = games[['seriesDescription','date','awayId','awayWinPct','awayScore']]

    # 创建一个 feature 用于判断是主场还是客场
    games_home['home'] = pd.Series()
    games_home['home'] = games_home['home'].fillna(1).astype(int)
    games_away['home'] = pd.Series()
    games_away['home'] = games_away['home'].fillna(0).astype(int)

    #为了后面主客场两站表的融合，改一下 columns name
    games_home.rename(columns={'homeId':'teamId',
                               'homeWinner':'Win',
                               'homeWinPct':'Current_winning_percentage',
                               'homeScore':'Score'},
                      inplace = True)
    games_away.rename(columns={'awayId':'teamId',
                               'awayWinner':'Win',
                               'awayWinPct':'Current_winning_percentage',
                               'awayScore':'Score'},
                      inplace = True)

    games_new = pd.concat([games_home,games_away],axis=0)

    index = pd.Series(list(range(0,games_new.shape[0])))       # 给整张表重置一下index
    games_new = games_new.set_index(index)
    
    playerBoxScores = merge_nested_form(data, 'playerBoxScores')

    # 改一下日期的 column name，方便后面 merge
    playerBoxScores.rename(columns={'gameDate':'date'},inplace=True)

    # 因为这张表计划大部分内容保留，所以只把没用的删除就好
    playerBoxScores.drop(columns = ['gameTimeUTC','teamName','playerName','home','gamePk'], inplace=True)
    
    teamBoxScores = merge_nested_form(data, 'teamBoxScores')

    # 改一下日期的 column name，方便后面 merge
    teamBoxScores.rename(columns={'gameDate':'date'},inplace=True)

    teamBoxScores.drop(['home','gamePk'], axis=1, inplace=True)
    
    standings = merge_nested_form(data, 'standings')

    # 改一下日期的 column name，方便后面 merge
    standings.rename(columns={'gameDate':'date'},inplace=True)
    
    standings_new = standings[['date','teamId','divisionRank','leagueRank',
                           'wins','losses','homeWins','homeLosses','awayWins','awayLosses']]

    standings_new.rename(columns={'wins':'totalWins_onTHEseason',
                                  'losses':'totalLosses_onTHEseason',
                                  'homeWins':'homeWins_onTHEseason',
                                  'homeLosses':'homeLosses_onTHEseason',
                                  'awayWins':'awayWins_onTHEseason',
                                  'awayLosses':'awayLosses_onTHEseason'},
                         inplace=True)
    
    awards = merge_nested_form(data, 'awards')
    # awards['awardId'].unique().shape[0] = 399
    # 一共有399种不同的奖项

    # 改一下日期的 column name，方便后面 merge
    awards.rename(columns={'awardDate':'date'},inplace=True)

    # 仅保留有用的几列
    if(awards.empty):
        awards['awardId'] = np.nan
        awards['date'] = merge_start['date']
        awards['playerId'] = merge_start['playerId']
    else:
        awards = awards[['awardId','date','playerId']]
    
#     transactions = merge_nested_form(data, 'transactions')
#     print("transactions:", transactions)
#     # transactions['typeDesc'].unique().shape[0] = 16
#     # 一共有16种不同的 transactions

#     # 嵌套表格以'date'为列名称的，数据格式好像全部都是 datetime，需要全部转换成 str
#     # 比如 transactions，playerTwitterFollowers，teamTwitterFollowers 表
#     # 但是以'gameDate'为列名称的，数据格式好像全部都是 str
#     transactions['date'] = transactions['date'].astype(str) 

#     # 有375行里，playerId 是 nan，就尼玛离谱，需要全部删除，然后再重置一下 index
#     delete_index = transactions.loc[transactions['playerId'].isnull()==True].index
#     transactions.drop(index = delete_index, inplace = True)
#     reset_index = pd.Series(list(range(0,transactions.shape[0])))    # 给整张表重置一下index
#     transactions = transactions.set_index(reset_index)

#     # 这个 playerId 默认有小数点就你妈离谱
#     transactions['playerId'] = transactions['playerId'].astype(int)

#     # transactions['toTeamId'].isnull().sum()==0
#     # 我们可以认为，没有球员被解聘，只存在两种情况：从别的队挖人or招募新人

#     transactions.drop(columns=['transactionId','playerName','fromTeamName','toTeamName','typeCode','description'],inplace=True)
    
    games_new_new = unique_record(games_new,'date','teamId','date_teamId')
    games_new_new.drop(columns='date_teamId',inplace=True)
    games_new_new['teamId'] = games_new_new['teamId'].astype(int)
    
    playerBoxScores_new = unique_record(playerBoxScores,'date','playerId','date_playerId')
    playerBoxScores_new.drop(columns='date_playerId',inplace=True)
    playerBoxScores_new['playerId'] = playerBoxScores_new['playerId'].astype(int)
    
    teamBoxScores_new = unique_record(teamBoxScores,'date','teamId','date_teamId')
    teamBoxScores_new.drop(columns='date_teamId',inplace=True)
    teamBoxScores_new['teamId'] = playerBoxScores_new['teamId'].astype(int)
    
    awards_new = unique_record(awards,'date','playerId','date_playerId')
    awards_new.drop(columns='date_playerId',inplace=True)
    awards_new['playerId'] = awards_new['playerId'].astype(int)
    
    gc.collect()
#     merge all
    merge_rosters = merge_two_nested_form(merge_start,rosters,key=['playerId','date'])
    gc.collect()
    merge_games = merge_two_nested_form(merge_rosters,games_new_new,key=['teamId','date'])
    gc.collect()
    merge_PScores = merge_two_nested_form(merge_games,playerBoxScores_new,key=['playerId','teamId','date'])
    gc.collect()
    merge_standings = merge_two_nested_form(merge_PScores,standings_new,key=['teamId','date'])
    gc.collect()
    merge_awards = merge_two_nested_form(merge_standings,awards_new,key=['playerId','date'])
    gc.collect()
    merge_final = merge_awards.copy()
    gc.collect()
    merge = merge_final
    
    merge['date'] = pd.to_datetime(merge['date'],format='%Y-%m-%d')
    merge['engagementMetricsDate'] = pd.to_datetime(merge['engagementMetricsDate'],format='%Y-%m-%d')
    
    df = merge.copy()
    df["playerId"] = df["playerId"].astype('str')
    df['playerId'] = df['playerId'].map(encode_dict["playerId"])
    df["teamId"] = df["teamId"].apply(trim_fraction)
    df['teamId'] = df['teamId'].map(encode_dict["teamId"])
    df["status"] = df["status"].astype('str')
    df['status'] = df['status'].map(encode_dict["status"])
    df["seriesDescription"] = df["seriesDescription"].astype('str')
    df['seriesDescription'] = df['seriesDescription'].map(encode_dict["seriesDescription"])
    df["positionName"] = df["positionName"].astype('str')
    df['positionName'] = df['positionName'].map(encode_dict["positionName"])
    df["positionType"] = df["positionType"].astype('str')
    df['positionType'] = df['positionType'].map(encode_dict["positionType"])
    df["awardId"] = df["awardId"].astype('str')
    df['awardId'] = df['awardId'].map(encode_dict["awardId"])
#     df.playerId.update(pd.Series(encode_dict["playerId"]))
    df["player_twitter"] = df["playerId"].map(ptt_dict)
    df["team_twitter"] = df["playerId"].map(ttt_dict)
    
#     dict_keys(['playerId', 'teamId', 'status', 'seriesDescription', 'positionName', 'positionType', 'awardId'])
    df = df.fillna(-1)
    
    df.drop(columns=["target1","target2","target3","target4","engagementMetricsDate","date"],inplace=True)
    pd.set_option('display.max_columns', None)
#     print(df.columns)
    df = df[X_columns]
    
#     print("df",df)
    return df

In [ ]:
env = mlb.make_env() # initialize the environment
iter_test = env.iter_test() # iterator which loops over each date in test set
test_data = pd.DataFrame()
sub_sample = pd.DataFrame()

In [ ]:
# for (test_df, sample_prediction_df) in iter_test: # make predictions here
#     sample_prediction_df = sample_prediction_df.reset_index(drop=True)
#     # creat dataset
#     sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
#                                         .map(lambda x: int(x.split('_')[1]))
#     sample_prediction_df['date'] = sample_prediction_df['date_playerId']\
#                                         .map(lambda x: int(x.split('_')[0]))
#     print(sample_prediction_df)

In [ ]:
for (test_df, sample_prediction_df) in iter_test: # make predictions here
    
    sample_prediction_df = sample_prediction_df.reset_index(drop=True)
    # creat dataset
    sample_prediction_df['playerId'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[1]))
    sample_prediction_df['date'] = sample_prediction_df['date_playerId']\
                                        .map(lambda x: int(x.split('_')[0]))
    
    
    # Dealing with missing values
    
    test_X = preprocess(test_df,sample_prediction_df)
#     test_data = pd.concat([test_data,test_df], axis=0)
#     sub_sample = pd.concat([sub_sample,sample_prediction_df], axis=0)
#     env.predict(sample_prediction_df)
    
#     # predict
    pred1 = model1.predict(test_X)
    pred2 = model2.predict(test_X)
    pred3 = model3.predict(test_X)
    pred4 = model4.predict(test_X)
    
    # merge submission
    sample_prediction_df['target1'] = np.clip(pred1, 0, 100)
    sample_prediction_df['target2'] = np.clip(pred2, 0, 100)
    sample_prediction_df['target3'] = np.clip(pred3, 0, 100)
    sample_prediction_df['target4'] = np.clip(pred4, 0, 100)
    sample_prediction_df = sample_prediction_df.fillna(0.)

#     sample_prediction_df["date"] = sample_prediction_df["date"].replace({"-":""}, regex=True)
#     sample_prediction_df["playerId"] = sample_prediction_df["playerId"].astype(str)
#     sample_prediction_df["date_playerId"] = sample_prediction_df["date"]+"_"+sample_prediction_df["playerId"]
    del sample_prediction_df['playerId']
    del sample_prediction_df['date']
#     sample_prediction_df = sample_prediction_df[["date_playerId","target1","target2","target3","target4"]]
    final_headers = ["date_playerId", "target1", "target2", "target3", "target4"] 
    sample_prediction_df = sample_prediction_df[final_headers].copy() 
#     sample_prediction_df["target1"] = sample_prediction_df["target1"].round(6)
#     sample_prediction_df["target2"] = sample_prediction_df["target2"].round(6)
#     sample_prediction_df["target3"] = sample_prediction_df["target3"].round(6)
#     sample_prediction_df["target4"] = sample_prediction_df["target4"].round(6)
    env.predict(sample_prediction_df)

In [ ]:
# test_data.to_csv("test_data2.csv",index = False)

In [ ]:
# sub_sample.to_csv("sub_sample.csv", index = False)

In [ ]:
sub = pd.read_csv("./submission.csv")
sub

In [ ]:
# sub.isnull().sum()

In [ ]:
# sub.dtypes

In [ ]:
# tmpsub

In [ ]:
# for i, v in sub.iterrows():
#     tmpsub.loc[tmpsub["date_playerId"]==v['date_playerId'],"target1"] = v["target1"]
#     tmpsub.loc[tmpsub["date_playerId"]==v['date_playerId'],"target2"] = v["target2"]
#     tmpsub.loc[tmpsub["date_playerId"]==v['date_playerId'],"target3"] = v["target3"]
#     tmpsub.loc[tmpsub["date_playerId"]==v['date_playerId'],"target4"] = v["target4"]

# tmpsub

In [ ]:
# tmpsub.to_csv('./submission.csv')